# Montu Python 
## Astronomical ephemerides for the Ancient World
## Test: Stellar catalogue


In [1]:
# Montu packages and data
from montu import *
import regex

# These magic commands are intended to allow updating the packages if they change
%load_ext autoreload
%autoreload 2

## Catalogues

We have compiled information on stars from the following sources:

- **Name of the stars**: File `IAU-CSN.json`.
  These are all stars approved by IAU. This include RA(J2000), Dec(J2000), mag. The file can be obtained from: https://raw.githubusercontent.com/mirandadam/iau-starnames/master/catalog_data/IAU-CSN.json. The source of this file is https://www.pas.rochester.edu/~emamajek/WGSN/IAU-CSN.txt. 

- **Astrometric information about most stars**: File `hip_catalogue.tsv`.
  Detailed astrometric information about most of the stars are provided by the Hipparcos catalogue.  This catalogue does not contains the information about all stars, for instance, the brightest stars. It comes from I/239/hip_main available at http://vizier.cds.unistra.fr/viz-bin/VizieR-3?-source=I/239/hip_main&-out.max=50&-out.form=HTML%20Table&-out.add=_r&-out.add=_RAJ,_DEJ&-sort=_r&-oc.form=sexa. 

- **Cross information**: Files `name.fab` (names of stars in Hipparcos catalogue), `cross_id.dat` (correspondence of HIP and SAO, HR, HD names).
  These files are obtained from Stellarium (folder `Resources/stars`).

- **Bright stars catalogue**. File `v50_catalogue.tsv`.
  This is the V/50/catalog available at https://vizier.cfa.harvard.edu/viz-bin/VizieR-3?-source=V/50/catalog. Contains 9110 stars, the brightest in the sky.

- **Compiled catalogue**: File `bright_stars.csv`.
  This is the original catalogue with wich we were working. I suspect it comes from a version of the HR catalogue

Other catalogues include:

- *Brightest stars catalogue*. 

- *Henry Draper catalogue*. III/135A, available at https://vizier.cfa.harvard.edu/viz-bin/VizieR?-source=III/135.

List of all proper names: https://en.wikipedia.org/wiki/List_of_proper_names_of_stars. 

Pay attention to this tool by astroquery: https://astroquery.readthedocs.io/en/latest/vizier/vizier.html.  

Constellations:

- Here you can learn to join stars with lines using the constellation file by Stellarium: https://levelup.gitconnected.com/how-to-use-python-to-create-custom-star-maps-for-your-next-stargazing-journey-9908b421f30e. 

## Desired Catalogue

We want to create a catalogue containing all stars with proper names, Bayer and Flamsteed designations. For those stars outside this range we want to include stars up to magnitude $V = 13$.   These are the columns we want to include:

In [2]:
stellar_catalogue_columns = [
    'HD', 'HR', 'HIP', # Catalogue designation, p.e. 
    'Name', # Official designation in the catalogue: proper name -> bayer name -> flamsteed name -> HD -> HR -> HIP
    'OtherDesinations', # All designations including different catalogues (Gliese, SAO), 
    'ProperName', # Proper name in UTF-8 and in ASCII, p.e. Arcturus 
    'Bayer','Flamsteed', # Bayer and Flamsteed designations, p.e.  α Boo, 
    'Constellation', # Abreviature of star in constellation, p.e. Boo
    'RAJ2000', 'DecJ2000', # Position in J2000 in degrees
    'pmRA', 'pmDec', 'vrad', # Proper motion in J2000 in mas/year, radial velocity in km/s
    'distance', 'parallax', # Distance an parallax
    'Vmag', 'B-V', # Apparent magnitude in V band and color index
    'SpType', # Spectral type
    'IsMultipleVisible','IsMultiplePhysical','IsVar', # Is multiple or variable
    'UpdateName', # Date of name update
]

## Reading the catalogues

### Names

In [3]:
names_cat = pd.read_json('catalogues/IAU-CSN.json')
print("All stars in catalogue = ",len(names_cat))
names_cat = names_cat[~names_cat.duplicated()]
print("Unique stars in catalogue = ",len(names_cat))
names_cat['HIP'] = names_cat['HIP'].str.replace('_','0').astype(int)
names_cat['HD'] = names_cat['HD'].str.replace('_','0').astype(int)

# Drop star without mag
names_cat['mag'] = names_cat['mag'].str.replace('_','NaN').astype(float)
names_cat = names_cat.dropna(subset=['mag'])

names_cat.rename(columns = {
    'Name/Diacritics': 'ProperName',
    'Con': 'Constellation',
    'Designation': 'OtherDesignations',
    'RA(J2000)': 'RAJ2000',
    'Dec(J2000)': 'DecJ2000',
    'Date': 'UpdateName',
    'mag': 'Vmag'
},inplace=True)

# Extract the HR designations
cond = names_cat.OtherDesignations.apply(lambda x:True if 'HR' in x else False)
HRs = names_cat.OtherDesignations[cond].apply(lambda x:x.split(' ')[1])
names_cat.loc[cond,'HR'] = HRs.astype(int)
names_cat.loc[~cond,'HR'] = 0
names_cat.HR = names_cat.HR.astype(int)

# Designation
names_cat['Name'] = names_cat['ProperName']

# Create Other designations
cond = (names_cat.ID != '_')
names_cat.OtherDesignations = names_cat.loc[cond].apply(
    lambda x:x['ID/Diacritics']+' '+x['Constellation']+'/'+x['OtherDesignations'],axis=1)

# Join ID and Con when Con is provided
columns = ['HD', 'HR', 'HIP', 'Name', 'OtherDesignations', 'ProperName', 'Constellation', 
           'RAJ2000', 'DecJ2000', 'Vmag', 'UpdateName']

names_cat = names_cat[columns]
names_cat.sort_values(by='Vmag',ascending=True,inplace=True)
names_cat.to_csv('catalogues/names_cat.csv',index=False)
names_cat

All stars in catalogue =  451
Unique stars in catalogue =  451


,HD,HR,HIP,Name,OtherDesignations,ProperName,Constellation,RAJ2000,DecJ2000,Vmag,UpdateName
381,48915,2491,32349,Sirius,α CMa/HR 2491,Sirius,CMa,101.28715499999999849,-16.71611599999999953,-1.44999999999999996,2016-06-30
116,45348,2326,30438,Canopus,α Car/HR 2326,Canopus,Car,95.98795800000000611,-52.69566100000000120,-0.62000000000000000,2016-06-30
76,124897,5340,69673,Arcturus,α Boo/HR 5340,Arcturus,Boo,213.91530000000000200,19.18240899999999982,-0.05000000000000000,2016-06-30
349,128620,5459,71683,Rigil Kentaurus,α Cen/HR 5459,Rigil Kentaurus,Cen,219.90206599999999071,-60.83397500000000235,-0.01000000000000000,2016-11-06
429,172167,7001,91262,Vega,α Lyr/HR 7001,Vega,Lyr,279.23473500000000058,38.78368900000000252,0.03000000000000000,2016-06-30
...,...,...,...,...,...,...,...,...,...,...,...
286,0,0,0,Moriah,NaN,Moriah,Del,306.12384800000000951,16.76217000000000112,12.16999999999999993,2019-12-17
285,0,0,0,Morava,NaN,Morava,Peg,356.66656100000000151,31.15593700000000155,12.17999999999999972,2019-12-17
326,0,0,0,Pipoltr,V1434 Her/TrES-3,Pipoltr,Her,268.02924400000000560,37.54617700000000013,12.19999999999999929,2019-12-17
199,0,0,0,Horna,NaN,Horna,Tri,35.38325100000000134,32.24613599999999991,12.30000000000000071,2019-12-17


### Denominations catalogue

In [4]:
# Read catalogue
denom_crcat = pd.read_csv('catalogues/name.fab',sep='|',header=None,names=['HIP','Denomination'])
print("All stars in catalogue = ",len(denom_crcat))
denom_crcat = denom_crcat[~denom_crcat.duplicated()]
print("Unique stars in catalogue = ",len(denom_crcat))
denom_crcat

# Change denomination
denom_crcat.rename(columns = {'Denomination':'Name'},inplace=True)

# Find the repeated HIPs
hip_repeated = []
hip_repeated_index = []
hip_ready = []
for index in tqdm.tqdm(denom_crcat.index):
    star = denom_crcat.loc[index]
    if star.HIP in hip_ready:
        hip_repeated += [star.HIP]
    hip_ready += [star.HIP]    

# Extract the duplicated
denom_duplicated = pd.DataFrame()
hip_ready = []
for hip in tqdm.tqdm(hip_repeated):
    if hip in hip_ready:
        continue
    repeated = denom_crcat[denom_crcat.HIP == hip]
    denom_duplicated = pd.concat([denom_duplicated,repeated])
    hip_ready += [hip]
    
# Set nonduplicated
denom_nonduplicated = pd.concat([denom_crcat,denom_duplicated]).drop_duplicates(keep=False)
denom_nonduplicated['OtherDesignations'] = denom_nonduplicated.Name
denom_nonduplicated['Name'] = denom_nonduplicated.Name
denom_nonduplicated['Flamsteed'] = ''
denom_nonduplicated['Bayer'] = ''
denom_nonduplicated['Constellation'] = ''
print("Entries non duplicated:",len(denom_nonduplicated))
print("Entries duplicated:",len(denom_duplicated))

# Set the other designations
stars = []
hip_ready = []
for index in tqdm.tqdm(denom_duplicated.index):
    star = denom_duplicated.loc[index]
    if star.HIP in hip_ready:
        continue
    cond = (denom_crcat.HIP == star.HIP)
    repeated = denom_crcat[cond]
    denoms = list(repeated.Name)
    
    all_denom = '/'.join(denoms)
    star['OtherDesignations'] = all_denom
    denomination = denoms[0]
    flamsteed = ''
    bayer = ''
    constellation = ''
    for den in denoms[::-1]:
        if regex.match(r'^[\p{Script=Greek}]+\d*_[a-zA-Z]+_*.*$',den):
            denomination = den
        if regex.match(r'^[\p{Script=Greek}]+\d*_[a-zA-Z]+_*.*$',den):
            bayer = den
            parts = bayer.split('_')
            constellation = parts[1]
        if regex.match(r'^\d*_[a-zA-Z]+$',den):
            flamsteed = den
            parts = flamsteed.split('_')
            constellation = parts[1]

    star['Bayer'] = bayer
    star['Flamsteed'] = flamsteed
    star['Name'] = denomination
    star['Constellation'] = constellation if re.match('^[a-zA-Z]{3}$',constellation) else ''
    hip_ready += [star.HIP]
    stars += [star]
denom_duplicated = pd.DataFrame(stars)
denom_duplicated['HasMultipleIDs'] = 1
print("Number of multiple HIP stars:",len(denom_duplicated))

# Set the nonduplicated
for index in tqdm.tqdm(denom_nonduplicated.index):
    #if int(index) != 86:continue
    star = denom_nonduplicated.loc[index]
    flamsteed = ''
    bayer = ''
    denomination = denom_nonduplicated.loc[index,'Name']
    if regex.match(r'^[\p{Script=Greek}]+\d*_[a-zA-Z]+_*.*$',denomination):
        bayer = denomination
        parts = bayer.split('_')
        constellation = parts[1]
    if regex.match(r'^\d*_[a-zA-Z]+_*.*$',denomination):
        flamsteed = denomination
        parts = flamsteed.split('_')
        constellation = parts[1]
    denom_nonduplicated.loc[index,'Bayer'] = bayer
    denom_nonduplicated.loc[index,'Flamsteed'] = flamsteed
    denom_nonduplicated.loc[index,'Constellation'] = constellation if re.match('^[a-zA-Z]{3}$',constellation) else ''
denom_nonduplicated['HasMultipleIDs'] = 0

# Merge duplicated and non duplicated
denom_crcat = pd.concat([denom_nonduplicated,denom_duplicated])

# Remove all "_"
for column in 'Name','OtherDesignations','Bayer','Flamsteed':
    denom_crcat[column] = denom_crcat[column].str.replace('_',' ')

# Save catalogue
denom_crcat.to_csv('catalogues/denom_crcat.csv',index=False)

denom_crcat

All stars in catalogue =  31141
Unique stars in catalogue =  31138


100%|██████████| 6438/6438 [00:01<00:00, 3707.09it/s]


Entries non duplicated: 20186
Entries duplicated: 10952


100%|██████████| 10952/10952 [00:06<00:00, 1616.13it/s]


Number of multiple HIP stars: 4514


100%|██████████| 20186/20186 [00:09<00:00, 2022.32it/s]


,HIP,Name,OtherDesignations,Flamsteed,Bayer,Constellation,HasMultipleIDs
2,841,22 And,22 And,22 And,,And,0
32,113919,3 And,3 And,3 And,,And,0
34,114210,5 And,5 And,5 And,,And,0
35,114430,6 And,6 And,6 And,,And,0
36,114570,7 And,7 And,7 And,,And,0
...,...,...,...,...,...,...,...
3673,118209,27 Psc,27 Psc/β 730,27 Psc,,Psc,1
31115,118213,RBR 55,RBR 55/RAO 76,,,,1
31122,118254,HJL 1113,HJL 1113/HO 207,,,,1
31135,118310,ALD 8,ALD 8/BAG 31,,,,1


In [6]:
(denom_crcat.Constellation != '').sum()

3698

### Cross IDs catalogue

In [7]:
id_crcat = pd.read_csv('catalogues/cross-id.dat',sep='\t').fillna(0)
print("All stars in catalogue = ",len(id_crcat))
id_crcat = id_crcat[~id_crcat.duplicated()]
print("Unique stars in catalogue = ",len(id_crcat))

for cat_field in '# HIP','SAO','HD','HR':
    id_crcat[cat_field] = id_crcat[cat_field].fillna(0).astype(int)

id_crcat.rename(columns = {'# HIP':'HIP', 'Unnamed: 1':'Component'},inplace=True)

columns = ['HD', 'HR', 'HIP', 'SAO', 'Component']
id_crcat = id_crcat[columns]

# Additional columns
id_crcat['IsMultiple'] = 0
id_crcat['Component'] = id_crcat.Component.astype(str)
cond = (id_crcat.Component!='0')
print('Multiple:',cond.sum())
id_crcat.loc[cond,'IsMultiple'] = 1
id_crcat.loc[~cond,'Component'] = ''

id_crcat.to_csv('catalogues/id_crcat.csv',index=False)
id_crcat

All stars in catalogue =  108378
Unique stars in catalogue =  108378
Multiple: 139


,HD,HR,HIP,SAO,Component,IsMultiple
0,224700,0,1,128522,,0
1,224690,0,2,165988,,0
2,224699,0,3,73664,,0
3,224707,0,4,248095,,0
4,224705,0,5,231883,,0
...,...,...,...,...,...,...
108373,224673,0,118318,108950,,0
108374,224693,0,118319,192301,,0
108375,224689,0,118320,128521,,0
108376,224685,0,118321,0,,0


In [8]:
cond = (id_crcat.Component!='')
Montu.print_df(id_crcat[cond].head(10))

,HD,HR,HIP,SAO,Component,IsMultiple
3541,4758,231,3885,74296,A,1
3542,4757,230,3885,74295,B,1
4273,5789,283,4675,36833,A,1
4274,5788,282,4675,36832,B,1
7098,10360,486,7751,232490,A,1
7099,10361,487,7751,0,B,1
8068,11503,546,8832,92681,A,1
8069,11502,545,8832,92680,B,1
8671,12447,596,9487,0,A,1
8672,12446,595,9487,0,B,1


## Merge denomination and ID catalogues

In [248]:
denom_crcat = pd.read_csv('catalogues/denom_crcat.csv')
id_crcat = pd.read_csv('catalogues/id_crcat.csv')

In [263]:
denom_id = pd.merge(denom_crcat,id_crcat,on='HIP',
                    suffixes=['_names','_denoms'],how='inner')

# Adjust catalogue names
for catalogue in 'HD', 'HR', 'HIP', 'SAO':
    denom_id[catalogue] = denom_id[catalogue].fillna(0).astype(int)

denom_id

,HIP,Name,OtherDesignations,Flamsteed,Bayer,Constellation,HasMultipleIDs,HD,HR,SAO,Component,IsMultiple
0,841,22 And,22 And,22 And,NaN,And,0,571,27,36123,NaN,0
1,113919,3 And,3 And,3 And,NaN,And,0,218031,8780,52649,NaN,0
2,114210,5 And,5 And,5 And,NaN,And,0,218470,8805,52713,NaN,0
3,114430,6 And,6 And,6 And,NaN,And,0,218804,8825,52761,NaN,0
4,114570,7 And,7 And,7 And,NaN,And,0,219080,8830,52787,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
23093,118205,B 2909,B 2909/RST 4136,NaN,NaN,NaN,1,224512,0,165975,NaN,0
23094,118209,27 Psc,27 Psc/β 730,27 Psc,NaN,Psc,1,224533,9067,147008,NaN,0
23095,118213,RBR 55,RBR 55/RAO 76,NaN,NaN,NaN,1,224531,0,73646,NaN,0
23096,118254,HJL 1113,HJL 1113/HO 207,NaN,NaN,NaN,1,224602,0,53549,NaN,0


In [250]:
cond = (~denom_id.Component.isna())
Montu.print_df(denom_id[cond].head(10))

,HIP,Name,OtherDesignations,Flamsteed,Bayer,Constellation,HasMultipleIDs,HD,HR,SAO,Component,IsMultiple
511,7751,p Eri,p Eri,NaN,NaN,Eri,0,10360,486,232490,A,1
512,7751,p Eri,p Eri,NaN,NaN,Eri,0,10361,487,0,B,1
534,21986,55 Eri,55 Eri,55 Eri,NaN,Eri,0,30021,1506,131443,A,1
535,21986,55 Eri,55 Eri,55 Eri,NaN,Eri,0,30020,1505,131442,B,1
2013,4675,Σ 79,Σ 79,NaN,NaN,NaN,0,5789,283,36833,A,1
2014,4675,Σ 79,Σ 79,NaN,NaN,NaN,0,5788,282,36832,B,1
6638,35960,HJ 3966,HJ 3966,NaN,NaN,NaN,0,58635,2843,197975,A,1
6639,35960,HJ 3966,HJ 3966,NaN,NaN,NaN,0,58634,2842,197974,B,1
6696,36345,Δ 49,Δ 49,NaN,NaN,NaN,0,59499,2870,198038,A,1
6697,36345,Δ 49,Δ 49,NaN,NaN,NaN,0,59500,2871,198039,B,1


HIP is not enough for merging. We need the HD number for each star. For that purpose we need another IDs. For instance pay attention to HIP 7751

In [251]:
denom_id[denom_id.HIP == 7751]

,HIP,Name,OtherDesignations,Flamsteed,Bayer,Constellation,HasMultipleIDs,HD,HR,SAO,Component,IsMultiple
511,7751,p Eri,p Eri,NaN,NaN,Eri,0,10360,486,232490,A,1
512,7751,p Eri,p Eri,NaN,NaN,Eri,0,10361,487,0,B,1


In [252]:
id = 'HIP'
cond = (denom_id[id] > 0)&(denom_id[id].duplicated())
repeated_HIP = denom_id[cond]
cond.sum()


72

In [253]:
Montu.print_df(repeated_HIP)

,HIP,Name,OtherDesignations,Flamsteed,Bayer,Constellation,HasMultipleIDs,HD,HR,SAO,Component,IsMultiple
512,7751,p Eri,p Eri,NaN,NaN,Eri,0,10361,487,0,B,1
535,21986,55 Eri,55 Eri,55 Eri,NaN,Eri,0,30020,1505,131442,B,1
2014,4675,Σ 79,Σ 79,NaN,NaN,NaN,0,5788,282,36832,B,1
6639,35960,HJ 3966,HJ 3966,NaN,NaN,NaN,0,58634,2842,197974,B,1
6697,36345,Δ 49,Δ 49,NaN,NaN,NaN,0,59500,2871,198039,B,1
6767,36817,H N 19,H N 19,NaN,NaN,NaN,0,60585,2910,174020,B,1
6782,36914,HJ 3997,HJ 3997,NaN,NaN,NaN,0,62154,2980,0,B,1
10376,61910,Σ 1669,Σ 1669,NaN,NaN,NaN,0,110317,4821,157447,B,1
11603,70327,Σ 1835,Σ 1835,NaN,NaN,NaN,0,126128,5385,0,B,1
13131,80399,H N 39,H N 39,NaN,NaN,NaN,0,147723,6106,184369,B,1


The following algorithm remove repetitions:

In [264]:
# Repeated ID
id = 'HIP'
cond = (denom_id[id] > 0)&(denom_id[id].duplicated())
repeated_HIP = denom_id[cond]

# Search, change other designations and remove repeated
indexes_to_remove = []
for i,index in enumerate(tqdm.tqdm(repeated_HIP.index)):
    star = repeated_HIP.loc[index]
    HIP = int(star.HIP)
    cond = (denom_id.HIP == HIP)
    #print(f"HIP {HIP} = {cond.sum()}")
    designations = f'{star.OtherDesignations}'
    for j,index_comp in enumerate(denom_id[cond].index):
        star_comp = denom_id[cond].loc[index_comp]
        if j==0:
            ibase = star_comp.name
        else:
            indexes_to_remove += [star_comp.name]
        if str(star_comp.Component) != 'nan':
            designations += f'/{star_comp.Name} {star_comp.Component}'
        for cat in 'HIP','HR','HD','SAO':
            if star_comp[cat]>0:
                designations += f'/{cat} {star_comp[cat]}'
        #print(star_comp.name,star_comp.HIP,star_comp.HD,star_comp.HR,star_comp.SAO,star_comp.Component,designations)
    designations = '/'.join(np.unique(designations.split('/')))
    denom_id.loc[ibase,'OtherDesignations'] = designations
    denom_id.loc[ibase,'HasMultipleIDs'] = 1
denom_id.drop(indexes_to_remove,inplace=True)
denom_id.to_csv('catalogues/denom_id.csv',index=False)

100%|██████████| 72/72 [00:00<00:00, 864.82it/s]


In [265]:
len(denom_id)

23026

In [266]:
denom_id[denom_id.HIP == 7751]

,HIP,Name,OtherDesignations,Flamsteed,Bayer,Constellation,HasMultipleIDs,HD,HR,SAO,Component,IsMultiple
511,7751,p Eri,HD 10360/HD 10361/HIP 7751/HR 486/HR 487/SAO 2...,NaN,NaN,Eri,1,10360,486,232490,A,1


## Merge names and denomination catalogues

In [267]:
names_cat = pd.read_csv('catalogues/names_cat.csv')
denom_id = pd.read_csv('catalogues/denom_id.csv')


In [268]:
names_cat

,HD,HR,HIP,Name,OtherDesignations,ProperName,Constellation,RAJ2000,DecJ2000,Vmag,UpdateName
0,48915,2491,32349,Sirius,α CMa/HR 2491,Sirius,CMa,101.28715499999999849,-16.71611599999999953,-1.44999999999999996,2016-06-30
1,45348,2326,30438,Canopus,α Car/HR 2326,Canopus,Car,95.98795800000000611,-52.69566100000000120,-0.62000000000000000,2016-06-30
2,124897,5340,69673,Arcturus,α Boo/HR 5340,Arcturus,Boo,213.91530000000000200,19.18240899999999982,-0.05000000000000000,2016-06-30
3,128620,5459,71683,Rigil Kentaurus,α Cen/HR 5459,Rigil Kentaurus,Cen,219.90206599999999071,-60.83397500000000235,-0.01000000000000000,2016-11-06
4,172167,7001,91262,Vega,α Lyr/HR 7001,Vega,Lyr,279.23473500000000058,38.78368900000000252,0.03000000000000000,2016-06-30
...,...,...,...,...,...,...,...,...,...,...,...
444,0,0,0,Moriah,NaN,Moriah,Del,306.12384800000000951,16.76217000000000112,12.16999999999999993,2019-12-17
445,0,0,0,Morava,NaN,Morava,Peg,356.66656100000000151,31.15593700000000155,12.17999999999999972,2019-12-17
446,0,0,0,Pipoltr,V1434 Her/TrES-3,Pipoltr,Her,268.02924400000000560,37.54617700000000013,12.19999999999999929,2019-12-17
447,0,0,0,Horna,NaN,Horna,Tri,35.38325100000000134,32.24613599999999991,12.30000000000000071,2019-12-17


In [269]:
(names_cat.HD > 0).sum(), (names_cat.HR > 0).sum(), (names_cat.HIP > 0).sum()

(406, 333, 411)

In [270]:
denom_id

,HIP,Name,OtherDesignations,Flamsteed,Bayer,Constellation,HasMultipleIDs,HD,HR,SAO,Component,IsMultiple
0,841,22 And,22 And,22 And,NaN,And,0,571,27,36123,NaN,0
1,113919,3 And,3 And,3 And,NaN,And,0,218031,8780,52649,NaN,0
2,114210,5 And,5 And,5 And,NaN,And,0,218470,8805,52713,NaN,0
3,114430,6 And,6 And,6 And,NaN,And,0,218804,8825,52761,NaN,0
4,114570,7 And,7 And,7 And,NaN,And,0,219080,8830,52787,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
23021,118205,B 2909,B 2909/RST 4136,NaN,NaN,NaN,1,224512,0,165975,NaN,0
23022,118209,27 Psc,27 Psc/β 730,27 Psc,NaN,Psc,1,224533,9067,147008,NaN,0
23023,118213,RBR 55,RBR 55/RAO 76,NaN,NaN,NaN,1,224531,0,73646,NaN,0
23024,118254,HJL 1113,HJL 1113/HO 207,NaN,NaN,NaN,1,224602,0,53549,NaN,0


Merge:

In [306]:
def merge_designations(desig1,desig2):
    desig1 = str(desig1)
    desig2 = str(desig2)
    merge = ''
    if (desig1 != 'nan') and (desig2 != 'nan'):
        merge = desig1+'/'+desig2
    else:
        merge = desig1 if (desig1 != 'nan') else desig2
    merge = '/'.join(sorted(np.unique(merge.split('/')[::-1])))
    return merge

In [281]:
merged = pd.merge(names_cat,denom_id,
                  on='HIP',
                  suffixes=['_names','_denoms'],
                  how='outer')

Montu.print_df(merged.head(5))
cat1 = 'names'
cat2 = 'denoms'
for field in ['Name','Constellation','HD','HR']:
    merged[f'{field}'] = merged[f'{field}_{cat1}'].fillna(merged[f'{field}_{cat2}'])
    print(f"Updating field {field}: {cat1}, {cat2}, updated = ",
          (~merged[f'{field}_{cat1}'].isnull()).sum(),
          (~merged[f'{field}_{cat2}'].isnull()).sum(),
          (~merged[f'{field}'].isnull()).sum())
#"""    
# Adjust catalogue names
for catalogue in 'HD', 'HR', 'HIP':
    merged[catalogue] = merged[catalogue].fillna(0).astype(int)

# Other designations
merged['OtherDesignations'] = merged.apply(
    lambda x:merge_designations(x.OtherDesignations_names,x.OtherDesignations_denoms),axis=1)

# Columns
columns = ['HD', 'HR', 'HIP', 'Name', 'OtherDesignations', 'ProperName', 'Bayer', 'Flamsteed',
           'Constellation','RAJ2000', 'DecJ2000', 'Vmag','IsMultiple','UpdateName']
merged = merged[columns]
alldenom_catalogue = merged
alldenom_catalogue.to_csv('catalogues/alldenom_catalogue.csv',index=False)
Montu.print_df(alldenom_catalogue.head(10))
alldenom_catalogue
#""";
#Montu.print_df(merged.head(5))

,HD_names,HR_names,HIP,Name_names,OtherDesignations_names,ProperName,Constellation_names,RAJ2000,DecJ2000,Vmag,UpdateName,Name_denoms,OtherDesignations_denoms,Flamsteed,Bayer,Constellation_denoms,HasMultipleIDs,HD_denoms,HR_denoms,SAO,Component,IsMultiple
0,48915.0,2491.0,32349,Sirius,α CMa/HR 2491,Sirius,CMa,101.28715499999999849,-16.71611599999999953,-1.44999999999999996,2016-06-30,α CMa,α CMa/9 CMa/AGC 1/LEC 1/HL 3/β 1411,9 CMa,α CMa,CMa,1.0,48915.0,2491.0,151881.0,NaN,0.0
1,45348.0,2326.0,30438,Canopus,α Car/HR 2326,Canopus,Car,95.98795800000000611,-52.69566100000000120,-0.62000000000000000,2016-06-30,α Car,α Car/MAM 2/CRU 9006,NaN,α Car,Car,1.0,45348.0,2326.0,234480.0,NaN,0.0
2,124897.0,5340.0,69673,Arcturus,α Boo/HR 5340,Arcturus,Boo,213.91530000000000200,19.18240899999999982,-0.05000000000000000,2016-06-30,α Boo,α Boo/16 Boo,16 Boo,α Boo,Boo,1.0,124897.0,5340.0,100944.0,NaN,0.0
3,128620.0,5459.0,71683,Rigil Kentaurus,α Cen/HR 5459,Rigil Kentaurus,Cen,219.90206599999999071,-60.83397500000000235,-0.01000000000000000,2016-11-06,α1 Cen,α1 Cen/LDS 494/RHD 1,NaN,α1 Cen,Cen,1.0,128620.0,5459.0,0.0,NaN,0.0
4,172167.0,7001.0,91262,Vega,α Lyr/HR 7001,Vega,Lyr,279.23473500000000058,38.78368900000000252,0.03000000000000000,2016-06-30,α Lyr,α Lyr/3 Lyr/Σ II 9/MBA 2/MET 11/H 5 39/SMR 33,3 Lyr,α Lyr,Lyr,1.0,172167.0,7001.0,67174.0,NaN,0.0


Updating field Name: names, denoms, updated =  449 23026 23135
Updating field Constellation: names, denoms, updated =  449 3685 3805
Updating field HD: names, denoms, updated =  449 23026 23135
Updating field HR: names, denoms, updated =  449 23026 23135


,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,RAJ2000,DecJ2000,Vmag,IsMultiple,UpdateName
0,48915,2491,32349,Sirius,9 CMa/AGC 1/HL 3/HR 2491/LEC 1/α CMa/β 1411,Sirius,α CMa,9 CMa,CMa,101.28715499999999849,-16.71611599999999953,-1.44999999999999996,0.0,2016-06-30
1,45348,2326,30438,Canopus,CRU 9006/HR 2326/MAM 2/α Car,Canopus,α Car,NaN,Car,95.98795800000000611,-52.69566100000000120,-0.62000000000000000,0.0,2016-06-30
2,124897,5340,69673,Arcturus,16 Boo/HR 5340/α Boo,Arcturus,α Boo,16 Boo,Boo,213.91530000000000200,19.18240899999999982,-0.05000000000000000,0.0,2016-06-30
3,128620,5459,71683,Rigil Kentaurus,HR 5459/LDS 494/RHD 1/α Cen/α1 Cen,Rigil Kentaurus,α1 Cen,NaN,Cen,219.90206599999999071,-60.83397500000000235,-0.01000000000000000,0.0,2016-11-06
4,172167,7001,91262,Vega,3 Lyr/H 5 39/HR 7001/MBA 2/MET 11/SMR 33/Σ II 9/α Lyr,Vega,α Lyr,3 Lyr,Lyr,279.23473500000000058,38.78368900000000252,0.03000000000000000,0.0,2016-06-30
5,34029,1708,24608,Capella,13 Aur/ANJ 1/BAR 25/FRH 1/HJ 2256/HR 1708/SHJ 51/SHY 23/SMR 26/α Aur/β 1392,Capella,α Aur,13 Aur,Aur,79.17232799999999315,45.99799099999999896,0.08000000000000000,0.0,2016-06-30
6,34085,1713,24436,Rigel,19 Ori/HR 1713/Σ 668/β 555/β Ori,Rigel,β Ori,19 Ori,Ori,78.63446700000000078,-8.20163800000000087,0.17999999999999999,0.0,2016-06-30
7,61421,2943,37279,Procyon,10 CMi/D 29/DIC 1/HR 2943/LAM 6/SHB 1/SLE 439/SMR 11/α CMi,Procyon,α CMi,10 CMi,CMi,114.82549299999999448,5.22499300000000044,0.40000000000000002,0.0,2016-06-30
8,10144,472,7588,Achernar,HR 472/α Eri,Achernar,α Eri,NaN,Eri,24.42852299999999843,-57.23675300000000021,0.45000000000000001,0.0,2016-06-30
9,39801,2061,27989,Betelgeuse,58 Ori/H 6 39/HR 2061/KAR 1/SLE 831/SMR 29/α Ori,Betelgeuse,α Ori,58 Ori,Ori,88.79293900000000406,7.40706400000000009,0.45000000000000001,0.0,2016-06-30


,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,RAJ2000,DecJ2000,Vmag,IsMultiple,UpdateName
0,48915,2491,32349,Sirius,9 CMa/AGC 1/HL 3/HR 2491/LEC 1/α CMa/β 1411,Sirius,α CMa,9 CMa,CMa,101.28715499999999849,-16.71611599999999953,-1.44999999999999996,0.0,2016-06-30
1,45348,2326,30438,Canopus,CRU 9006/HR 2326/MAM 2/α Car,Canopus,α Car,NaN,Car,95.98795800000000611,-52.69566100000000120,-0.62000000000000000,0.0,2016-06-30
2,124897,5340,69673,Arcturus,16 Boo/HR 5340/α Boo,Arcturus,α Boo,16 Boo,Boo,213.91530000000000200,19.18240899999999982,-0.05000000000000000,0.0,2016-06-30
3,128620,5459,71683,Rigil Kentaurus,HR 5459/LDS 494/RHD 1/α Cen/α1 Cen,Rigil Kentaurus,α1 Cen,NaN,Cen,219.90206599999999071,-60.83397500000000235,-0.01000000000000000,0.0,2016-11-06
4,172167,7001,91262,Vega,3 Lyr/H 5 39/HR 7001/MBA 2/MET 11/SMR 33/Σ II ...,Vega,α Lyr,3 Lyr,Lyr,279.23473500000000058,38.78368900000000252,0.03000000000000000,0.0,2016-06-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23130,224512,0,118205,B 2909,B 2909/RST 4136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
23131,224533,9067,118209,27 Psc,27 Psc/β 730,NaN,NaN,27 Psc,Psc,NaN,NaN,NaN,0.0,NaN
23132,224531,0,118213,RBR 55,RAO 76/RBR 55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
23133,224602,0,118254,HJL 1113,HJL 1113/HO 207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


Check that no HIP duplicates are present:

In [287]:
id = 'HIP'
cond = (alldenom_catalogue[id] > 0)&(alldenom_catalogue[id].duplicated())
repeated = alldenom_catalogue[cond]
cond.sum()

0

## V50 Catalogue

Preview:

In [288]:
v50_cat = pd.read_csv('catalogues/v50_catalogue.tsv',sep=';',header=88)
v50_cat

,_Glon,_Glat,_RAJ2000,_DEJ2000,_Elon2000,_Elat2000,HR,Name,HD,FK5,...,GLAT,Vmag,B-V,SpType,pmRA,pmDE,Parallax,RadVel,Dmag,Sep
0,114.444683,-16.878665,001.291250,+45.229167,022.867788,+40.168262,1,,3,,...,-16.88,6.70,0.07,A1Vn,-0.012,-0.018,,-18,4.2,21.60000000000000142
1,098.327534,-61.139797,001.265833,-0.503056,000.961320,-0.965043,2,,6,,...,-61.14,6.29,1.10,gG9,0.045,-0.060,,14,,NaN
2,093.754038,-65.932553,001.333750,-5.707500,358.945562,-5.765455,3,33 Psc,28,1002,...,-65.93,4.61,1.04,K0IIIbCN-0.5,-0.009,0.089,0.014,-6,2.5,0.00000000000000000
3,106.186507,-47.982972,001.425000,+13.396111,006.706500,+11.708996,4,86 Peg,87,2004,...,-47.98,5.51,0.90,G5III,0.045,-0.012,,-2,,NaN
4,117.034840,-3.916935,001.566667,+58.436667,033.934291,+50.901155,5,,123,,...,-3.92,5.96,0.67,G5V,0.263,0.030,0.047,-12,0.8,1.39999999999999991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9105,307.683594,-43.794914,001.127917,-72.897778,308.107507,-61.547830,9106,,225233,,...,-43.80,7.31,0.44,F2V,0.010,-0.054,,8,,NaN
9106,112.178535,-27.238331,001.224167,+34.659722,016.419661,+30.982969,9107,,225239,2002,...,-27.24,6.12,0.62,G2V,0.772,0.089,0.035,4,,NaN
9107,308.184066,-45.211542,001.172083,-71.436944,310.618122,-60.731632,9108,,225253,1001,...,-45.21,5.59,-0.12,B8IV-V,0.028,-0.014,,-3,,NaN
9108,110.218368,-35.072165,001.233333,+26.648889,012.375468,+23.819615,9109,,225276,,...,-35.07,6.25,1.40,K4IIIb,0.110,-0.012,,-5,4.5,17.50000000000000000


Columns:

In [289]:
v50_cat.columns

Index(['_Glon', '_Glat', '_RAJ2000', '_DEJ2000', '_Elon2000', '_Elat2000',
       'HR', 'Name', 'HD', 'FK5', 'ADS', 'VarID', 'RAJ2000', 'DEJ2000', 'GLON',
       'GLAT', 'Vmag', 'B-V', 'SpType', 'pmRA', 'pmDE', 'Parallax', 'RadVel',
       'Dmag', 'Sep'],
      dtype='object')

Functions to process data:

In [21]:
def hex2dec(d,m,s):
    sgn = np.sign(float(d))
    sgn = +1 if sgn==0 else sgn
    return sgn*(float(abs(float(d)))+float(m)/60+float(s)/3600)

def generate_designations(row,catalogues):
    designation = row.OtherDesignations
    for cat in catalogues:
        catalogue_name = str(row[cat])
        if re.match('\s*\d+\s*',catalogue_name):
            designation += f"/{cat} {catalogue_name}"
    designation = designation.lstrip('/')
    designation = designation.rstrip('/')
    return designation

def strip_blanks(str):
    stripped = re.sub(r'\s{2,}',' ',str).rstrip().lstrip()
    return stripped

Process data:

In [290]:
v50_cat = pd.read_csv('catalogues/v50_catalogue.tsv',sep=';',header=88)
print("All stars in catalogue = ",len(v50_cat))
v50_cat = v50_cat[~v50_cat.duplicated()]
Montu.print_df(v50_cat.head(5))

# Catalogue fields as integers
v50_cat['Name'] = v50_cat['Name'].apply(strip_blanks)
v50_cat['HD'] = v50_cat['HD'].apply(strip_blanks)
v50_cat['HD'] = v50_cat['HD'].replace('',0).astype(int)
v50_cat['HR'] = v50_cat['HR'].astype(int)
    
# Remove blank spaces
v50_cat.replace(r'^\s+$',np.nan,regex=True,inplace=True)

# Drop RA and Dec
v50_cat['RAJ2000'] = v50_cat.RAJ2000.replace(r'^\s+$',np.nan,regex=True)
v50_cat.dropna(subset=['RAJ2000'],inplace=True)
print("Final stars in catalogue = ",len(v50_cat))

# Rename columns
v50_cat.rename(columns = {'RAJ2000': 'RAhms', 'DEJ2000': 'DEdms', 'pmDE':'pmDec'}, inplace=True)

# Convert RA and Dec to decimal
v50_cat['RAJ2000'] = v50_cat.RAhms.apply(lambda x:15*hex2dec(*x.split(' ')))
v50_cat['DecJ2000'] = v50_cat.DEdms.apply(lambda x:hex2dec(*x.split(' ')))

# Create catalogue names
for cat in 'HD','HR':
    cond = (v50_cat[cat] > 0)
    v50_cat.loc[cond,f'{cat}_Name'] = v50_cat.loc[cond,cat].apply(lambda x:f'{cat} '+str(x))

# Create Other designations
v50_cat['OtherDesignations'] = v50_cat['Name']
v50_cat['OtherDesignations'] = v50_cat.apply(lambda x:generate_designations(x,['HD','HR']),axis=1)

# Create name based on catalogue
v50_cat['Name'] = v50_cat.HD_Name.fillna(v50_cat.HR_Name)

# Check if multiple or variable
v50_cat['IsMultiple'] = 0
v50_cat.loc[~np.isnan(v50_cat.Sep),'IsMultiple'] = 1

v50_cat['IsVariable'] = 0
v50_cat.loc[~(v50_cat.VarID.isna()),'IsVariable'] = 1

# Adjust units
v50_cat['pmRA'] = v50_cat['pmRA'].apply(lambda x:1000*float(x)) # arcsec/year -> mas/year
v50_cat['pmDec'] = v50_cat['pmDec'].apply(lambda x:1000*float(x)) # arcsec/year -> mas/year
v50_cat['Parallax'] = v50_cat['Parallax'].apply(lambda x:1000*float(x)) # arcsec/year -> mas/year

# Columns
columns = ['HD', 'HR', 'Name','OtherDesignations', 
           'RAJ2000', 'DecJ2000', 'pmRA', 'pmDec', 
           'Vmag', 'B-V', 'SpType', 
           'Parallax', 'RadVel', 
           'IsMultiple', 'IsVariable']
v50_cat = v50_cat[columns]

v50_cat.to_csv('catalogues/v50_cat.csv',index=False)
Montu.print_df(v50_cat.head(10))

All stars in catalogue =  9110


,_Glon,_Glat,_RAJ2000,_DEJ2000,_Elon2000,_Elat2000,HR,Name,HD,FK5,ADS,VarID,RAJ2000,DEJ2000,GLON,GLAT,Vmag,B-V,SpType,pmRA,pmDE,Parallax,RadVel,Dmag,Sep
0,114.444683,-16.878665,001.291250,+45.229167,022.867788,+40.168262,1,,3,,46,,00 05 09.9,+45 13 45,114.44,-16.88,6.70,0.07,A1Vn,-0.012,-0.018,,-18,4.2,21.60000000000000142
1,098.327534,-61.139797,001.265833,-0.503056,000.961320,-0.965043,2,,6,,,,00 05 03.8,-00 30 11,98.33,-61.14,6.29,1.10,gG9,0.045,-0.060,,14,,NaN
2,093.754038,-65.932553,001.333750,-5.707500,358.945562,-5.765455,3,33 Psc,28,1002,,Var?,00 05 20.1,-05 42 27,93.75,-65.93,4.61,1.04,K0IIIbCN-0.5,-0.009,0.089,0.014,-6,2.5,0.00000000000000000
3,106.186507,-47.982972,001.425000,+13.396111,006.706500,+11.708996,4,86 Peg,87,2004,,,00 05 42.0,+13 23 46,106.19,-47.98,5.51,0.90,G5III,0.045,-0.012,,-2,,NaN
4,117.034840,-3.916935,001.566667,+58.436667,033.934291,+50.901155,5,,123,,61,V640 Cas,00 06 16.0,+58 26 12,117.03,-3.92,5.96,0.67,G5V,0.263,0.030,0.047,-12,0.8,1.39999999999999991


Final stars in catalogue =  9096


,HD,HR,Name,OtherDesignations,RAJ2000,DecJ2000,pmRA,pmDec,Vmag,B-V,SpType,Parallax,RadVel,IsMultiple,IsVariable
0,3,1,HD 3,HD 3/HR 1,1.29125000000000001,45.22916666666667140,-12.0,-18.0,6.70,0.07,A1Vn,NaN,-18,1,0
1,6,2,HD 6,HD 6/HR 2,1.26583333333333337,0.50305555555555559,45.0,-60.0,6.29,1.10,gG9,NaN,14,0,0
2,28,3,HD 28,33 Psc/HD 28/HR 3,1.33374999999999977,-5.70750000000000046,-9.0,89.0,4.61,1.04,K0IIIbCN-0.5,14.0,-6,1,1
3,87,4,HD 87,86 Peg/HD 87/HR 4,1.42500000000000004,13.39611111111111086,45.0,-12.0,5.51,0.90,G5III,NaN,-2,0,0
4,123,5,HD 123,HD 123/HR 5,1.56666666666666687,58.43666666666666032,263.0,30.0,5.96,0.67,G5V,47.0,-12,1,1
5,142,6,HD 142,HD 142/HR 6,1.57916666666666683,-49.07500000000000284,565.0,-38.0,5.70,0.52,G1IV,50.0,3,1,0
6,144,7,HD 144,10 Cas/HD 144/HR 7,1.61041666666666683,64.19611111111110802,8.0,0.0,5.59,-0.03,B9III,NaN,0,0,0
7,166,8,HD 166,HD 166/HR 8,1.65333333333333332,29.02138888888888957,380.0,-182.0,6.13,0.75,K0V,67.0,-8,1,1
8,203,9,HD 203,HD 203/HR 9,1.70874999999999999,-23.10750000000000171,100.0,-45.0,6.18,0.38,A7V,NaN,3,0,0
9,256,10,HD 256,HD 256/HR 10,1.82583333333333320,-17.38638888888888800,-18.0,36.0,6.19,0.14,A6Vn,NaN,-9,0,0


We need to recall the structure:

```python
stellar_catalogue_columns = [
    'HD', 'HR', 'HIP', # Catalogue designation, p.e. 
    'Name', # Official designation in the catalogue: proper name -> bayer name -> flamsteed name -> HD -> HR -> HIP
    'OtherDesinations', # All designations including different catalogues (Gliese, SAO), 
    'ProperName', # Proper name in UTF-8 and in ASCII, p.e. Arcturus 
    'Bayer','Flamsteed', # Bayer and Flamsteed designations, p.e.  α Boo, 
    'Constellation', # Abreviature of star in constellation, p.e. Boo
    'RAJ2000', 'DecJ2000', # Position in J2000 in degrees
    'pmRA', 'pmDec', 'vrad', # Proper motion in J2000 in mas/year, radial velocity in km/s
    'distance', 'parallax', # Distance an parallax
    'Vmag', 'B-V', # Apparent magnitude in V band and color index
    'SpType', # Spectral type
    'IsMultipleVisible','IsMultiplePhysical','IsVar', # Is multiple or variable
    'UpdateName', # Date of name update
]
```

## Merge denominations and V50

In [291]:
alldenom_catalogue = pd.read_csv('catalogues/alldenom_catalogue.csv')
v50_cat = pd.read_csv('catalogues/v50_cat.csv')

In [292]:
alldenom_catalogue

,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,RAJ2000,DecJ2000,Vmag,IsMultiple,UpdateName
0,48915,2491,32349,Sirius,9 CMa/AGC 1/HL 3/HR 2491/LEC 1/α CMa/β 1411,Sirius,α CMa,9 CMa,CMa,101.28715499999999849,-16.71611599999999953,-1.44999999999999996,0.0,2016-06-30
1,45348,2326,30438,Canopus,CRU 9006/HR 2326/MAM 2/α Car,Canopus,α Car,NaN,Car,95.98795800000000611,-52.69566100000000120,-0.62000000000000000,0.0,2016-06-30
2,124897,5340,69673,Arcturus,16 Boo/HR 5340/α Boo,Arcturus,α Boo,16 Boo,Boo,213.91530000000000200,19.18240899999999982,-0.05000000000000000,0.0,2016-06-30
3,128620,5459,71683,Rigil Kentaurus,HR 5459/LDS 494/RHD 1/α Cen/α1 Cen,Rigil Kentaurus,α1 Cen,NaN,Cen,219.90206599999999071,-60.83397500000000235,-0.01000000000000000,0.0,2016-11-06
4,172167,7001,91262,Vega,3 Lyr/H 5 39/HR 7001/MBA 2/MET 11/SMR 33/Σ II ...,Vega,α Lyr,3 Lyr,Lyr,279.23473500000000058,38.78368900000000252,0.03000000000000000,0.0,2016-06-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23130,224512,0,118205,B 2909,B 2909/RST 4136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
23131,224533,9067,118209,27 Psc,27 Psc/β 730,NaN,NaN,27 Psc,Psc,NaN,NaN,NaN,0.0,NaN
23132,224531,0,118213,RBR 55,RAO 76/RBR 55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
23133,224602,0,118254,HJL 1113,HJL 1113/HO 207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [293]:
v50_cat

,HD,HR,Name,OtherDesignations,RAJ2000,DecJ2000,pmRA,pmDec,Vmag,B-V,SpType,Parallax,RadVel,IsMultiple,IsVariable
0,3,1,HD 3,HD 3/HR 1,1.29125000000000001,45.22916666666667140,-12.0,-18.0,6.70000000000000018,0.07000000000000001,A1Vn,NaN,-18.0,1,0
1,6,2,HD 6,HD 6/HR 2,1.26583333333333337,0.50305555555555559,45.0,-60.0,6.29000000000000004,1.10000000000000009,gG9,NaN,14.0,0,0
2,28,3,HD 28,33 Psc/HD 28/HR 3,1.33374999999999977,-5.70750000000000046,-9.0,89.0,4.61000000000000032,1.04000000000000004,K0IIIbCN-0.5,14.0,-6.0,1,1
3,87,4,HD 87,86 Peg/HD 87/HR 4,1.42500000000000004,13.39611111111111086,45.0,-12.0,5.50999999999999979,0.90000000000000002,G5III,NaN,-2.0,0,0
4,123,5,HD 123,HD 123/HR 5,1.56666666666666687,58.43666666666666032,263.0,30.0,5.95999999999999996,0.67000000000000004,G5V,47.0,-12.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9091,225233,9106,HD 225233,HD 225233/HR 9106,1.12791666666666690,-72.89777777777779022,10.0,-54.0,7.30999999999999961,0.44000000000000000,F2V,NaN,8.0,0,0
9092,225239,9107,HD 225239,HD 225239/HR 9107,1.22416666666666663,34.65972222222222143,772.0,89.0,6.12000000000000011,0.62000000000000000,G2V,35.0,4.0,0,0
9093,225253,9108,HD 225253,HD 225253/HR 9108,1.17208333333333314,-71.43694444444444969,28.0,-14.0,5.58999999999999986,-0.12000000000000000,B8IV-V,NaN,-3.0,0,0
9094,225276,9109,HD 225276,HD 225276/HR 9109,1.23333333333333339,26.64888888888888729,110.0,-12.0,6.25000000000000000,1.39999999999999991,K4IIIb,NaN,-5.0,1,0


In [307]:
def merge_designations(desig1,desig2):
    desig1 = str(desig1)
    desig2 = str(desig2)
    merge = ''
    if (desig1 != 'nan') and (desig2 != 'nan'):
        merge = desig1+'/'+desig2
    else:
        merge = desig1 if (desig1 != 'nan') else desig2
    merge = '/'.join(sorted(np.unique(merge.split('/')[::-1])))
    return merge

def strip_blanks_protected(instr):
    instr_str = str(instr)
    if instr_str=='nan':
        stripped='N/A'
    else:
        stripped = re.sub(r'\s{2,}',' ',instr_str).rstrip().lstrip()
    return stripped

In [308]:
cat1 = 'denoms'
cat2 = 'v50'
merged = pd.merge(alldenom_catalogue,v50_cat,
                  on=['HR','HD'],
                  suffixes=[f'_{cat1}',f'_{cat2}'],
                  how='outer')
Montu.print_df(merged.head(5))

for field in ['Vmag','RAJ2000','DecJ2000','Name','IsMultiple']:
    merged[f'{field}'] = merged[f'{field}_{cat1}'].fillna(merged[f'{field}_{cat2}'])
    print(f"Updating field {field}: {cat1}, {cat2}, updated = ",
          (~merged[f'{field}_{cat1}'].isnull()).sum(),
          (~merged[f'{field}_{cat2}'].isnull()).sum(),
          (~merged[f'{field}'].isnull()).sum())

merged['OtherDesignations'] = merged.apply(
    lambda x:merge_designations(x[f'OtherDesignations_{cat1}'],x[f'OtherDesignations_{cat2}']),axis=1)

# Adjust catalogue names
for catalogue in 'HD', 'HR', 'HIP':
    merged[catalogue] = merged[catalogue].fillna(0).astype(int)

# Add MN catalogue number
merged.sort_values(by=['Vmag','RAJ2000'],inplace=True)
merged.reset_index(inplace=True)
merged['MN'] = merged.apply(lambda x:int(x.name)+1,axis=1)

# Fill IsMultiple and IsVar
merged['IsMultiple'] = merged['IsMultiple'].fillna(0)
merged['IsVariable'] = merged['IsVariable'].fillna(0)

# Fill spectral type 
merged['SpType'] = merged['SpType'].apply(strip_blanks_protected)

# Columns
columns = ['MN','HD', 'HR', 'HIP', 'Name', 'OtherDesignations', 'ProperName', 'Bayer', 'Flamsteed','Constellation',
           'RAJ2000', 'DecJ2000', 'pmRA', 'pmDec', 'RadVel', 'Parallax',
           'Vmag','B-V','SpType','IsMultiple','IsVariable',
           'UpdateName']
merged = merged[columns]

print("Catalogue completeness:")
nstars = len(merged)
for col in columns:
    ncomplete = (~(merged[col].isnull())).sum()
    print(f"\t{col}: {ncomplete}/{nstars} ({ncomplete/nstars*100:.1f}%)")

print("Number of stars:",len(merged))
stellar_catalogue_denom = merged
stellar_catalogue_denom.to_csv('catalogues/stellar_catalogue_denom.csv',index=False)
stellar_catalogue_denom
Montu.print_df(stellar_catalogue_denom.head(5))
#Montu.print_df(merged.head(5))

,HD,HR,HIP,Name_denoms,OtherDesignations_denoms,ProperName,Bayer,Flamsteed,Constellation,RAJ2000_denoms,DecJ2000_denoms,Vmag_denoms,IsMultiple_denoms,UpdateName,Name_v50,OtherDesignations_v50,RAJ2000_v50,DecJ2000_v50,pmRA,pmDec,Vmag_v50,B-V,SpType,Parallax,RadVel,IsMultiple_v50,IsVariable
0,48915,2491,32349.0,Sirius,9 CMa/AGC 1/HL 3/HR 2491/LEC 1/α CMa/β 1411,Sirius,α CMa,9 CMa,CMa,101.28715499999999849,-16.71611599999999953,-1.44999999999999996,0.0,2016-06-30,HD 48915,9Alp CMa/HD 48915/HR 2491,101.28708333333331382,-16.71611111111111114,-553.0,-1205.0,-1.45999999999999996,0.00000000000000000,A1Vm,375.0,-8.0,1.0,0.0
1,45348,2326,30438.0,Canopus,CRU 9006/HR 2326/MAM 2/α Car,Canopus,α Car,NaN,Car,95.98795800000000611,-52.69566100000000120,-0.62000000000000000,0.0,2016-06-30,HD 45348,Alp Car/HD 45348/HR 2326,95.98791666666667766,-52.69583333333333286,22.0,21.0,-0.71999999999999997,0.14999999999999999,F0II,28.0,21.0,0.0,0.0
2,124897,5340,69673.0,Arcturus,16 Boo/HR 5340/α Boo,Arcturus,α Boo,16 Boo,Boo,213.91530000000000200,19.18240899999999982,-0.05000000000000000,0.0,2016-06-30,HD 124897,16Alp Boo/HD 124897/HR 5340,213.91541666666668675,19.18250000000000099,-1093.0,-1998.0,-0.04000000000000000,1.22999999999999998,K1.5IIIFe-0.5,90.0,-5.0,0.0,1.0
3,128620,5459,71683.0,Rigil Kentaurus,HR 5459/LDS 494/RHD 1/α Cen/α1 Cen,Rigil Kentaurus,α1 Cen,NaN,Cen,219.90206599999999071,-60.83397500000000235,-0.01000000000000000,0.0,2016-11-06,HD 128620,Alp1Cen/HD 128620/HR 5459,219.89958333333331097,-60.83527777777778311,-3642.0,699.0,-0.01000000000000000,0.70999999999999996,G2V,751.0,-22.0,1.0,0.0
4,172167,7001,91262.0,Vega,3 Lyr/H 5 39/HR 7001/MBA 2/MET 11/SMR 33/Σ II 9/α Lyr,Vega,α Lyr,3 Lyr,Lyr,279.23473500000000058,38.78368900000000252,0.03000000000000000,0.0,2016-06-30,HD 172167,3Alp Lyr/HD 172167/HR 7001,279.23458333333337578,38.78361111111110660,202.0,286.0,0.03000000000000000,0.00000000000000000,A0Va,123.0,-14.0,1.0,1.0


Updating field Vmag: denoms, v50, updated =  449 9105 9224
Updating field RAJ2000: denoms, v50, updated =  449 9105 9224
Updating field DecJ2000: denoms, v50, updated =  449 9105 9224
Updating field Name: denoms, v50, updated =  23135 9105 27044
Updating field IsMultiple: denoms, v50, updated =  23026 9105 26939
Catalogue completeness:
	MN: 27044/27044 (100.0%)
	HD: 27044/27044 (100.0%)
	HR: 27044/27044 (100.0%)
	HIP: 27044/27044 (100.0%)
	Name: 27044/27044 (100.0%)
	OtherDesignations: 27044/27044 (100.0%)
	ProperName: 449/27044 (1.7%)
	Bayer: 1712/27044 (6.3%)
	Flamsteed: 2828/27044 (10.5%)
	Constellation: 3805/27044 (14.1%)
	RAJ2000: 9224/27044 (34.1%)
	DecJ2000: 9224/27044 (34.1%)
	pmRA: 9105/27044 (33.7%)
	pmDec: 9105/27044 (33.7%)
	RadVel: 9099/27044 (33.6%)
	Parallax: 3294/27044 (12.2%)
	Vmag: 9224/27044 (34.1%)
	B-V: 8794/27044 (32.5%)
	SpType: 27044/27044 (100.0%)
	IsMultiple: 27044/27044 (100.0%)
	IsVariable: 27044/27044 (100.0%)
	UpdateName: 449/27044 (1.7%)
Number of stars: 

,MN,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,RAJ2000,DecJ2000,pmRA,pmDec,RadVel,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,UpdateName
0,1,48915,2491,32349,Sirius,9 CMa/9Alp CMa/AGC 1/HD 48915/HL 3/HR 2491/LEC 1/α CMa/β 1411,Sirius,α CMa,9 CMa,CMa,101.28715499999999849,-16.71611599999999953,-553.0,-1205.0,-8.0,375.0,-1.44999999999999996,0.00000000000000000,A1Vm,0.0,0.0,2016-06-30
1,2,45348,2326,30438,Canopus,Alp Car/CRU 9006/HD 45348/HR 2326/MAM 2/α Car,Canopus,α Car,NaN,Car,95.98795800000000611,-52.69566100000000120,22.0,21.0,21.0,28.0,-0.62000000000000000,0.14999999999999999,F0II,0.0,0.0,2016-06-30
2,3,124897,5340,69673,Arcturus,16 Boo/16Alp Boo/HD 124897/HR 5340/α Boo,Arcturus,α Boo,16 Boo,Boo,213.91530000000000200,19.18240899999999982,-1093.0,-1998.0,-5.0,90.0,-0.05000000000000000,1.22999999999999998,K1.5IIIFe-0.5,0.0,1.0,2016-06-30
3,4,128620,5459,71683,Rigil Kentaurus,Alp1Cen/HD 128620/HR 5459/LDS 494/RHD 1/α Cen/α1 Cen,Rigil Kentaurus,α1 Cen,NaN,Cen,219.90206599999999071,-60.83397500000000235,-3642.0,699.0,-22.0,751.0,-0.01000000000000000,0.70999999999999996,G2V,0.0,0.0,2016-11-06
4,5,172167,7001,91262,Vega,3 Lyr/3Alp Lyr/H 5 39/HD 172167/HR 7001/MBA 2/MET 11/SMR 33/Σ II 9/α Lyr,Vega,α Lyr,3 Lyr,Lyr,279.23473500000000058,38.78368900000000252,202.0,286.0,-14.0,123.0,0.03000000000000000,0.00000000000000000,A0Va,0.0,1.0,2016-06-30


## Missing astrometric information: HIPPARCOS Catalogue

In [309]:
hip_cat = pd.read_csv('catalogues/hip_catalogue.tsv',sep='\t',header=94)
Montu.print_df(hip_cat.head(5))

,_RAJ2000,_DEJ2000,_RAB1950,_DEB1950,_Elon2000,_Elat2000,HIP,RAhms,DEdms,Vmag,RAICRS,DEICRS,Plx,pmRA,pmDE,e_Plx,e_pmRA,e_pmDE,B-V,Period,HvarType,MultFlag,rho,Notes,HD,BD,SpType
0,000.0008992088,+01.0890087506,359.3603142912,+00.8106290353,000.4340516791,+00.9987787998,1,00 00 00.22,+01 05 20.4,9.10,000.00091185,+01.08901332,3.54,-5.20,-1.88,1.39,1.36,0.81,0.482,,,,,,224700,B+00 5077,F5
1,000.0042646079,-19.4988397098,359.3603577026,-19.7772649067,351.9863984169,-17.8347189112,2,00 00 00.91,-19 29 55.8,9.27,000.00379737,-19.49883745,21.90,181.21,-0.93,3.10,1.74,0.92,0.999,,C,O,,,224690,B-20 6688,K3V
2,000.0050243058,+38.8592790071,359.3654683997,+38.5809718453,017.7747932987,+35.1423666843,3,00 00 01.20,+38 51 33.4,6.61,000.00500795,+38.85928608,2.81,5.24,-2.91,0.63,0.57,0.47,-0.019,,C,,,,224699,B+38 5108,B9
3,000.0086292358,-51.8935457309,359.3645647432,-52.1720255616,333.1127964736,-46.2175300834,4,00 00 02.01,-51 53 36.8,8.06,000.00838170,-51.89354612,7.75,62.85,0.16,0.97,0.65,0.65,0.370,,,,,,224707,,F0V
4,000.0099734379,-40.5912023549,359.3679371716,-40.8697945628,341.1875314854,-36.6567161863,5,00 00 02.39,-40 35 28.4,8.55,000.00996534,-40.59122440,2.87,2.53,9.07,1.11,0.67,0.74,0.902,,,,,,224705,,G8III


Columns:

In [310]:
hip_cat.columns

Index(['_RAJ2000', '_DEJ2000', '_RAB1950', '_DEB1950', '_Elon2000',
       '_Elat2000', 'HIP', 'RAhms', 'DEdms', 'Vmag', 'RAICRS', 'DEICRS', 'Plx',
       'pmRA', 'pmDE', 'e_Plx', 'e_pmRA', 'e_pmDE', 'B-V', 'Period',
       'HvarType', 'MultFlag', 'rho', 'Notes', 'HD', 'BD', 'SpType'],
      dtype='object')

Functions to process data:

In [311]:
def hex2dec(d,m,s):
    sgn = np.sign(float(d))
    sgn = +1 if sgn==0 else sgn
    return sgn*(float(abs(float(d)))+float(m)/60+float(s)/3600)

def generate_designations(row,catalogues):
    designation = row.OtherDesignations
    for cat in catalogues:
        catalogue_name = str(row[cat])
        if catalogue_name!='0' and catalogue_name!='nan' and re.match('\s*\w+\s*',catalogue_name):
            designation += f"/{cat} {catalogue_name}"
    designation = designation.lstrip('/')
    designation = designation.rstrip('/')
    return designation

def strip_blanks(str):
    stripped = re.sub(r'\s{2,}',' ',str).rstrip().lstrip()
    return stripped

def strip_blanks_protected(instr):
    instr_str = str(instr)
    if instr_str=='nan':
        stripped='N/A'
    else:
        stripped = re.sub(r'\s{2,}',' ',instr_str).rstrip().lstrip()
    return stripped

Process data:

In [312]:
hip_cat = pd.read_csv('catalogues/hip_catalogue.tsv',sep='\t',header=94)
#hip_cat = hip_cat.iloc[:100]
print("All stars in catalogue = ",len(hip_cat))
hip_cat = hip_cat[~hip_cat.duplicated()]

# Catalogue fields as integers
hip_cat['SpType'] = hip_cat['SpType'].apply(strip_blanks_protected)
hip_cat['HD'] = hip_cat['HD'].apply(strip_blanks)
hip_cat['HD'] = hip_cat['HD'].replace('',0).astype(int)

# Remove blank spaces
hip_cat.replace(r'^\s+$',np.nan,regex=True,inplace=True)

# Create catalogue names
for cat in 'HD','HIP':
    cond = (hip_cat[cat] > 0)
    hip_cat.loc[cond,f'{cat}_Name'] = hip_cat.loc[cond,cat].apply(lambda x:f'{cat} '+str(x))

# Create Name and other designations
hip_cat['Name'] = hip_cat['HD_Name'].fillna(hip_cat['HIP_Name'])
hip_cat['OtherDesignations'] = ''
hip_cat['OtherDesignations'] = hip_cat.apply(lambda x:generate_designations(x,['HD','HIP','BD']),axis=1)

# Drop blank RA and Dec
hip_cat['RAhms'] = hip_cat.RAhms.replace(r'^\s+$',np.nan,regex=True)
hip_cat.dropna(subset=['RAhms'],inplace=True)
print("Stars with astrometric data in catalogue = ",len(hip_cat))

# Convert RA and Dec to decimal
hip_cat['RAJ2000'] = hip_cat.RAhms.apply(lambda x:15*hex2dec(*x.split(' ')))
hip_cat['DecJ2000'] = hip_cat.DEdms.apply(lambda x:hex2dec(*x.split(' ')))

# Check if multiple or variable
hip_cat['IsMultiple'] = 0
hip_cat.loc[~hip_cat.Period.isnull(),'IsMultiple'] = 1
hip_cat.loc[~hip_cat.MultFlag.isnull(),'IsMultiple'] = 1

hip_cat['IsVariable'] = 0
hip_cat.loc[~hip_cat.HvarType.isnull(),'IsVariable'] = 1

# Rename Columns
hip_cat.rename(columns = {
    'pmDE':'pmDec', 'Plx':'Parallax',
    }, inplace=True)

columns = ['HD', 'HIP', 'Name', 'OtherDesignations', 
           'RAJ2000', 'DecJ2000', 'pmRA', 'pmDec', 'Parallax',
           'Vmag','B-V','SpType','IsMultiple','IsVariable','BD']

hip_cat = hip_cat[columns]
hip_cat.to_csv('catalogues/hip_cat.csv',index=False)
Montu.print_df(hip_cat.head(10))

All stars in catalogue =  118218
Stars with astrometric data in catalogue =  118218


,HD,HIP,Name,OtherDesignations,RAJ2000,DecJ2000,pmRA,pmDec,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,BD
0,224700,1,HD 224700,HD 224700/HIP 1/BD B+00 5077,0.00091666666666667,1.08899999999999997,-5.20,-1.88,3.54,9.10,0.482,F5,0,0,B+00 5077
1,224690,2,HD 224690,HD 224690/HIP 2/BD B-20 6688,0.00379166666666667,-19.49883333333333368,181.21,-0.93,21.90,9.27,0.999,K3V,1,1,B-20 6688
2,224699,3,HD 224699,HD 224699/HIP 3/BD B+38 5108,0.00500000000000000,38.85927777777777692,5.24,-2.91,2.81,6.61,-0.019,B9,0,1,B+38 5108
3,224707,4,HD 224707,HD 224707/HIP 4,0.00837500000000000,-51.89355555555555810,62.85,0.16,7.75,8.06,0.370,F0V,0,0,NaN
4,224705,5,HD 224705,HD 224705/HIP 5,0.00995833333333333,-40.59122222222222121,2.53,9.07,2.87,8.55,0.902,G8III,0,0,NaN
5,0,6,HIP 6,HIP 6,0.01812500000000000,3.94650000000000034,226.29,-12.84,18.80,12.31,1.336,M0V:,0,0,NaN
6,0,7,HIP 7,HIP 7/BD B+19 5185,0.02254166666666667,20.03661111111111381,-208.12,-200.79,17.74,9.64,0.740,G0,0,1,B+19 5185
7,224709,8,HD 224709,HD 224709/HIP 8/BD B+25 5054,0.02729166666666667,25.88647222222222055,19.09,-5.66,5.17,9.05,1.102,M6e-M8.5e Tc,1,1,B+25 5054
8,224708,9,HD 224708,HD 224708/HIP 9/BD B+35 5149,0.03533333333333333,36.58594444444444349,-6.30,8.42,4.81,8.59,1.067,G5,0,1,B+35 5149
9,224717,10,HD 224717,HD 224717/HIP 10,0.03625000000000000,-50.86708333333333343,42.23,40.02,10.76,8.59,0.489,F6V,0,0,NaN


## Final merge: the Montu catalogue

In [313]:
stellar_catalogue_denom = pd.read_csv('catalogues/stellar_catalogue_denom.csv')
hip_cat = pd.read_csv('catalogues/hip_cat.csv')

In [314]:
stellar_catalogue_denom

,MN,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,...,pmRA,pmDec,RadVel,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,UpdateName
0,1,48915,2491,32349,Sirius,9 CMa/9Alp CMa/AGC 1/HD 48915/HL 3/HR 2491/LEC...,Sirius,α CMa,9 CMa,CMa,...,-553.0,-1205.0,-8.0,375.0,-1.44999999999999996,0.00000000000000000,A1Vm,0.0,0.0,2016-06-30
1,2,45348,2326,30438,Canopus,Alp Car/CRU 9006/HD 45348/HR 2326/MAM 2/α Car,Canopus,α Car,NaN,Car,...,22.0,21.0,21.0,28.0,-0.62000000000000000,0.14999999999999999,F0II,0.0,0.0,2016-06-30
2,3,124897,5340,69673,Arcturus,16 Boo/16Alp Boo/HD 124897/HR 5340/α Boo,Arcturus,α Boo,16 Boo,Boo,...,-1093.0,-1998.0,-5.0,90.0,-0.05000000000000000,1.22999999999999998,K1.5IIIFe-0.5,0.0,1.0,2016-06-30
3,4,128620,5459,71683,Rigil Kentaurus,Alp1Cen/HD 128620/HR 5459/LDS 494/RHD 1/α Cen/...,Rigil Kentaurus,α1 Cen,NaN,Cen,...,-3642.0,699.0,-22.0,751.0,-0.01000000000000000,0.70999999999999996,G2V,0.0,0.0,2016-11-06
4,5,172167,7001,91262,Vega,3 Lyr/3Alp Lyr/H 5 39/HD 172167/HR 7001/MBA 2/...,Vega,α Lyr,3 Lyr,Lyr,...,202.0,286.0,-14.0,123.0,0.03000000000000000,0.00000000000000000,A0Va,0.0,1.0,2016-06-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27039,27040,224233,0,118010,OSO 209,LEP 117/OSO 209,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
27040,27041,224512,0,118205,B 2909,B 2909/RST 4136,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
27041,27042,224531,0,118213,RBR 55,RAO 76/RBR 55,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
27042,27043,224602,0,118254,HJL 1113,HJL 1113/HO 207,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN


In [315]:
hip_cat

,HD,HIP,Name,OtherDesignations,RAJ2000,DecJ2000,pmRA,pmDec,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,BD
0,224700,1,HD 224700,HD 224700/HIP 1/BD B+00 5077,0.00091666666666660,1.08899999999999997,-5.20000000000000018,-1.87999999999999989,3.54000000000000004,9.09999999999999964,0.48199999999999998,F5,0,0,B+00 5077
1,224690,2,HD 224690,HD 224690/HIP 2/BD B-20 6688,0.00379166666666660,-19.49883333333333368,181.21000000000000796,-0.93000000000000005,21.89999999999999858,9.26999999999999957,0.99900000000000000,K3V,1,1,B-20 6688
2,224699,3,HD 224699,HD 224699/HIP 3/BD B+38 5108,0.00500000000000000,38.85927777777777692,5.24000000000000021,-2.91000000000000014,2.81000000000000005,6.61000000000000032,-0.01900000000000000,B9,0,1,B+38 5108
3,224707,4,HD 224707,HD 224707/HIP 4,0.00837500000000000,-51.89355555555555810,62.85000000000000142,0.16000000000000000,7.75000000000000000,8.06000000000000050,0.37000000000000000,F0V,0,0,NaN
4,224705,5,HD 224705,HD 224705/HIP 5,0.00995833333333330,-40.59122222222222121,2.52999999999999980,9.07000000000000028,2.87000000000000011,8.55000000000000071,0.90200000000000002,G8III,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118213,224673,118318,HD 224673,HD 224673/HIP 118318/BD B+10 5018,359.96375000000000455,11.67372222222222078,-2.16000000000000014,2.08999999999999986,1.91999999999999993,6.99000000000000021,1.59499999999999997,K2,0,1,B+10 5018
118214,224693,118319,HD 224693,HD 224693/HIP 118319,359.97391666666669607,-22.42816666666666947,148.74000000000000909,27.53000000000000114,10.63000000000000078,8.23000000000000043,0.63900000000000001,G2V,0,1,NaN
118215,224689,118320,HD 224689,HD 224689/HIP 118320/BD B+05 5245,359.97604166666678793,5.95663888888888948,20.92000000000000171,-35.25999999999999801,5.00000000000000000,7.58999999999999986,0.99900000000000000,K0,0,1,B+05 5245
118216,224685,118321,HD 224685,HD 224685/HIP 118321,359.97825000000000273,-64.37258333333332416,216.99000000000000909,106.45999999999999375,19.21999999999999886,9.19999999999999929,0.69799999999999995,G5V,0,0,NaN


In [316]:
def merge_designations(desig1,desig2):
    desig1 = str(desig1)
    desig2 = str(desig2)
    merge = ''
    if (desig1 != 'nan') and (desig2 != 'nan'):
        merge = desig1+'/'+desig2
    else:
        merge = desig1 if (desig1 != 'nan') else desig2
    merge = '/'.join(sorted(np.unique(merge.split('/')[::-1])))
    return merge

Let's merge

In [320]:
cat1 = 'denoms'
cat2 = 'hip'
merged = pd.merge(stellar_catalogue_denom,hip_cat,
                  on=['HIP'], #,'HD'],
                  suffixes=[f'_{cat1}',f'_{cat2}'],
                  how='outer')

# Adjust catalogue names
for catalogue in 'MN', 'HR', 'HIP':
    merged[catalogue] = merged[catalogue].fillna(0).astype(int)

# Drop all non-MN stars
cond = (merged.MN > 0)
merged = merged[cond]

# Merged other designations
merged['OtherDesignations'] = merged.apply(
    lambda x:merge_designations(x[f'OtherDesignations_{cat1}'],x[f'OtherDesignations_{cat2}']),axis=1)

# Merge properties: priority is HIP
cat_pri = cat2
cat_sec = cat1
for field in ['Vmag','RAJ2000','DecJ2000','Parallax','pmRA','pmDec','IsMultiple','IsVariable','B-V','HD']:
    merged[f'{field}'] = merged[f'{field}_{cat_pri}'].fillna(merged[f'{field}_{cat_sec}'])
    print(f"Updating field {field}: {cat_pri}, {cat_sec}, updated = ",
          (~merged[f'{field}_{cat_pri}'].isnull()).sum(),
          (~merged[f'{field}_{cat_sec}'].isnull()).sum(),
          (~merged[f'{field}'].isnull()).sum())

# Merge properties: priority is denom
cat_pri = cat1
cat_sec = cat2
for field in ['Name','SpType']:
    merged[f'{field}'] = merged[f'{field}_{cat_pri}'].fillna(merged[f'{field}_{cat_sec}'])
    print(f"Updating field {field}: {cat_pri}, {cat_sec}, updated = ",
          (~merged[f'{field}_{cat_pri}'].isnull()).sum(),
          (~merged[f'{field}_{cat_sec}'].isnull()).sum(),
          (~merged[f'{field}'].isnull()).sum())
    
# Adjust integer fields
for field in 'HD', 'IsMultiple', 'IsVariable':
    merged[field] = merged[field].fillna(0).astype(int)

# Add MN catalogue number
merged.sort_values(by=['Vmag','RAJ2000'],inplace=True)
merged.reset_index(inplace=True)
merged['MN'] = merged.apply(lambda x:int(x.name)+1,axis=1)
merged['MN_Name'] = merged['MN'].apply(lambda x:f'MN {x}')
merged['OtherDesignations'] = merged.apply(
    lambda x:merge_designations(x[f'OtherDesignations'],x[f'MN_Name']),axis=1)

# Output columns
columns = ['MN','HD', 'HR', 'HIP', 'Name', 'OtherDesignations', 'ProperName', 'Bayer', 'Flamsteed','Constellation',
           'RAJ2000', 'DecJ2000', 'pmRA', 'pmDec', 'RadVel', 'Parallax',
           'Vmag','B-V','SpType','IsMultiple','IsVariable',
           'UpdateName']
merged = merged[columns]

print("Catalogue completeness:")
nstars = len(merged)
for col in columns:
    ncomplete = (~(merged[col].isnull())).sum()
    print(f"\t{col}: {ncomplete}/{nstars} ({ncomplete/nstars*100:.1f}%)")

# Fill empty physical variables
for column in ['Parallax','B-V','RadVel','pmRA','pmDec']:
    merged[column] = merged[column].fillna(0)

merged['SpType'] = merged['SpType'].apply(strip_blanks_protected)

# Store catalogue
montu_catalogue = merged
montu_catalogue.to_csv('catalogues/montu_catalogue.csv',index=False)
montu_catalogue
Montu.print_df(montu_catalogue.head(5))

print("Final number of stars:",len(merged))

Updating field Vmag: hip, denoms, updated =  23097 9224 27044
Updating field RAJ2000: hip, denoms, updated =  23097 9224 27044
Updating field DecJ2000: hip, denoms, updated =  23097 9224 27044
Updating field Parallax: hip, denoms, updated =  23094 3294 23671
Updating field pmRA: hip, denoms, updated =  23094 9105 27006
Updating field pmDec: hip, denoms, updated =  23094 9105 27006
Updating field IsMultiple: hip, denoms, updated =  23097 27044 27044
Updating field IsVariable: hip, denoms, updated =  23097 27044 27044
Updating field B-V: hip, denoms, updated =  22895 8794 26604
Updating field HD: hip, denoms, updated =  23097 27044 27044
Updating field Name: denoms, hip, updated =  27044 23097 27044
Updating field SpType: denoms, hip, updated =  9105 23032 26942
Catalogue completeness:
	MN: 27044/27044 (100.0%)
	HD: 27044/27044 (100.0%)
	HR: 27044/27044 (100.0%)
	HIP: 27044/27044 (100.0%)
	Name: 27044/27044 (100.0%)
	OtherDesignations: 27044/27044 (100.0%)
	ProperName: 449/27044 (1.7%)
	

,MN,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,RAJ2000,DecJ2000,pmRA,pmDec,RadVel,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,UpdateName
0,1,48915,2491,32349,Sirius,9 CMa/9Alp CMa/AGC 1/BD B-16 1591 /HD 48915/HIP 32349/HL 3/HR 2491/LEC 1/MN 1/α CMa/β 1411,Sirius,α CMa,9 CMa,CMa,101.28854166666666003,-16.71313888888888854,-546.00999999999999091,-1223.07999999999992724,-8.0,379.20999999999997954,-1.43999999999999995,0.00900000000000000,A1Vm,1,1,2016-06-30
1,2,45348,2326,30438,Canopus,Alp Car/CRU 9006/HD 45348/HIP 30438/HR 2326/MAM 2/MN 2/α Car,Canopus,α Car,NaN,Car,95.98787500000000250,-52.69572222222221569,19.98999999999999844,23.67000000000000171,21.0,10.42999999999999972,-0.62000000000000000,0.16400000000000001,F0II,0,1,2016-06-30
2,3,124897,5340,69673,Arcturus,16 Boo/16Alp Boo/BD B+19 2777 /HD 124897/HIP 69673/HR 5340/MN 3/α Boo,Arcturus,α Boo,16 Boo,Boo,213.91812500000000341,19.18727777777777632,-1093.45000000000004547,-1999.40000000000009095,-5.0,88.84999999999999432,-0.05000000000000000,1.23900000000000010,K1.5IIIFe-0.5,1,1,2016-06-30
3,4,128620,5459,71683,Rigil Kentaurus,Alp1Cen/HD 128620/HIP 71683/HR 5459/LDS 494/MN 4/RHD 1/α Cen/α1 Cen,Rigil Kentaurus,α1 Cen,NaN,Cen,219.92041666666665378,-60.83513888888889198,-3678.19000000000005457,481.83999999999997499,-22.0,742.12000000000000455,-0.01000000000000000,0.70999999999999996,G2V,1,1,2016-11-06
4,5,172167,7001,91262,Vega,3 Lyr/3Alp Lyr/BD B+38 3238 /H 5 39/HD 172167/HIP 91262/HR 7001/MBA 2/MET 11/MN 5/SMR 33/Σ II 9/α Lyr,Vega,α Lyr,3 Lyr,Lyr,279.23412500000000591,38.78300000000000125,201.02000000000001023,287.45999999999997954,-14.0,128.93000000000000682,0.03000000000000000,-0.00100000000000000,A0Va,0,1,2016-06-30


Final number of stars: 27044


In [321]:
montu_catalogue

,MN,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,...,pmRA,pmDec,RadVel,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,UpdateName
0,1,48915,2491,32349,Sirius,9 CMa/9Alp CMa/AGC 1/BD B-16 1591 /HD 48915/HI...,Sirius,α CMa,9 CMa,CMa,...,-546.00999999999999091,-1223.07999999999992724,-8.0,379.20999999999997954,-1.43999999999999995,0.00900000000000000,A1Vm,1,1,2016-06-30
1,2,45348,2326,30438,Canopus,Alp Car/CRU 9006/HD 45348/HIP 30438/HR 2326/MA...,Canopus,α Car,NaN,Car,...,19.98999999999999844,23.67000000000000171,21.0,10.42999999999999972,-0.62000000000000000,0.16400000000000001,F0II,0,1,2016-06-30
2,3,124897,5340,69673,Arcturus,16 Boo/16Alp Boo/BD B+19 2777 /HD 124897/HIP 6...,Arcturus,α Boo,16 Boo,Boo,...,-1093.45000000000004547,-1999.40000000000009095,-5.0,88.84999999999999432,-0.05000000000000000,1.23900000000000010,K1.5IIIFe-0.5,1,1,2016-06-30
3,4,128620,5459,71683,Rigil Kentaurus,Alp1Cen/HD 128620/HIP 71683/HR 5459/LDS 494/MN...,Rigil Kentaurus,α1 Cen,NaN,Cen,...,-3678.19000000000005457,481.83999999999997499,-22.0,742.12000000000000455,-0.01000000000000000,0.70999999999999996,G2V,1,1,2016-11-06
4,5,172167,7001,91262,Vega,3 Lyr/3Alp Lyr/BD B+38 3238 /H 5 39/HD 172167/...,Vega,α Lyr,3 Lyr,Lyr,...,201.02000000000001023,287.45999999999997954,-14.0,128.93000000000000682,0.03000000000000000,-0.00100000000000000,A0Va,0,1,2016-06-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27039,27040,0,0,0,Moriah,MN 27040,Moriah,NaN,NaN,Del,...,0.00000000000000000,0.00000000000000000,0.0,0.00000000000000000,12.16999999999999993,0.00000000000000000,N/A,0,0,2019-12-17
27040,27041,0,0,0,Morava,MN 27041,Morava,NaN,NaN,Peg,...,0.00000000000000000,0.00000000000000000,0.0,0.00000000000000000,12.17999999999999972,0.00000000000000000,N/A,0,0,2019-12-17
27041,27042,0,0,0,Pipoltr,MN 27042/TrES-3/V1434 Her,Pipoltr,NaN,NaN,Her,...,0.00000000000000000,0.00000000000000000,0.0,0.00000000000000000,12.19999999999999929,0.00000000000000000,N/A,0,0,2019-12-17
27042,27043,0,0,0,Horna,MN 27043,Horna,NaN,NaN,Tri,...,0.00000000000000000,0.00000000000000000,0.0,0.00000000000000000,12.30000000000000071,0.00000000000000000,N/A,0,0,2019-12-17


## Statistics

In [342]:
cond_pn = (~montu_catalogue.ProperName.isnull())
print("Number of stars with proper name: ",cond_pn.sum())
cond_by = (~montu_catalogue.Bayer.isnull())
print("Number of stars with Bayer name: ",cond_by.sum())
cond_fm = (~montu_catalogue.Flamsteed.isnull())
print("Number of stars with Flamsteed name: ",cond_fm.sum())
cond_all = (cond_pn)&(cond_by)&(cond_fm)
print("Number of proper, Bayer and Flamsteed: ",cond_all.sum())

Number of stars with proper name:  449
Number of stars with Bayer name:  1712
Number of stars with Flamsteed name:  2828
Number of proper, Bayer and Flamsteed:  260


In [349]:
cond = (montu_catalogue.Vmag<5.5)
nvis = len(np.unique(montu_catalogue.loc[cond,'HD']))
print("Number of naked eye stars: ",nvis)

Number of naked eye stars:  2842


## Basic checks

In [331]:
id = 'HIP'
cond = (montu_catalogue[id] > 0)&(montu_catalogue[id].duplicated())
repeated = montu_catalogue[cond]
cond.sum()

0

In [332]:
id = 'HD'
cond = (montu_catalogue[id] > 0)&(montu_catalogue[id].duplicated())
repeated = montu_catalogue[cond]
cond.sum()
Montu.print_df(repeated)

,MN,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,RAJ2000,DecJ2000,pmRA,pmDec,RadVel,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,UpdateName
414,415,138917,5789,76276,δ Ser,13 Ser/13Del Ser/BD B+11 2821 /HD 138917/HD 138918/HIP 76276/HR 5788/HR 5789/MN 415/SAO 101623/SAO 101624/SMR 62/Σ 1954/δ Ser/δ Ser A/δ Ser B,NaN,δ Ser,13 Ser,Ser,233.70079166666667447,10.53886111111111035,-72.62999999999999545,3.16999999999999993,-42.0,15.53999999999999915,3.79999999999999982,0.26800000000000002,F0IV,1,1,NaN
497,498,17878,854,0,HD 17878,18Tau Per/HD 17878/HR 854/MN 498,NaN,NaN,NaN,NaN,43.56458333333333144,52.76250000000000284,0.00000000000000000,-5.00000000000000000,2.0,19.00000000000000000,3.95000000000000018,0.73999999999999999,G4III+A4V,1,1,NaN
518,519,192909,7751,0,HD 192909,32 Cyg/HD 192909/HR 7751/MN 519,NaN,NaN,NaN,NaN,303.86791666666670153,47.71444444444444599,1.00000000000000000,8.00000000000000000,-14.0,14.00000000000000000,3.97999999999999998,1.52000000000000002,K3Ib+B3V,1,1,NaN
801,802,213310,8572,0,HD 213310,5 Lac/HD 213310/HR 8572/MN 802,NaN,NaN,NaN,NaN,337.38249999999999318,47.70694444444444571,2.00000000000000000,0.00000000000000000,-4.0,6.00000000000000000,4.36000000000000032,1.67999999999999994,M0II+B8V,0,1,NaN
926,927,61555,2948,0,HD 61555,HD 61555/HR 2948/MN 927,NaN,NaN,NaN,NaN,114.70541666666663616,-26.80166666666666231,-17.00000000000000000,19.00000000000000000,24.0,10.00000000000000000,4.50000000000000000,-0.17000000000000001,B6V,1,1,NaN
1026,1027,213051,8558,0,HD 213051,55Zet1Aqr/HD 213051/HR 8558/MN 1027,NaN,NaN,NaN,NaN,337.20708333333334394,0.02027777777777770,181.00000000000000000,9.00000000000000000,29.0,22.00000000000000000,4.58999999999999986,0.00000000000000000,F6IV,1,0,NaN
1059,1060,18519,888,13914,ε Ari,48 Ari/48Eps Ari/BD B+20 484 /HD 18519/HD 18520/HIP 13914/HR 887/HR 888/MN 1060/SAO 75673/Σ 333/ε Ari/ε Ari A/ε Ari B,NaN,ε Ari,48 Ari,Ari,44.80304166666666532,21.34044444444444366,-13.03999999999999915,-6.54999999999999982,-8.0,11.15000000000000036,4.62999999999999989,0.04800000000000000,A2V s,1,1,NaN
1168,1169,218640,8817,114375,c3 Aqr,89 Aqr/HD 218640/HIP 114375/MN 1169/RST 3320/c3 Aqr,NaN,NaN,89 Aqr,Aqr,347.47866666666664059,-22.45758333333332857,35.92000000000000171,-7.79000000000000004,0.0,6.25999999999999979,4.70999999999999996,0.67400000000000004,A3IV:,1,1,NaN
1351,1352,129246,5477,0,HD 129246,30Zet Boo/HD 129246/HR 5477/MN 1352,NaN,NaN,NaN,NaN,220.28708333333332803,13.72833333333333172,53.00000000000000000,-16.00000000000000000,-6.0,9.00000000000000000,4.83000000000000007,0.00000000000000000,A2III,1,1,NaN
1413,1414,98230,4374,0,HD 98230,53Xi UMa/HD 98230/HR 4374/MN 1414,NaN,NaN,NaN,NaN,169.54541666666668220,31.52916666666666501,-432.00000000000000000,-586.00000000000000000,-16.0,137.00000000000000000,4.87000000000000011,0.00000000000000000,G0V,1,1,NaN


In [333]:
Montu.print_df(montu_catalogue[montu_catalogue.HD == 219175])

,MN,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,RAJ2000,DecJ2000,pmRA,pmDec,RadVel,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,UpdateName
13399,13400,219175,0,114702,Σ 2993,BD B-09 6149 /HD 219175/HIP 114702/MN 13400/Σ 2993,NaN,NaN,NaN,NaN,348.52979166666671063,-8.92424999999999891,551.25999999999999091,-37.82000000000000028,0.0,26.51999999999999957,7.54999999999999982,0.54400000000000004,F9V,1,1,NaN
17594,17595,219175,0,114703,S 826,BD B-09 6150 /HD 219175/HIP 114703/MN 17595/S 826,NaN,NaN,NaN,NaN,348.53033333333337396,-8.93119444444444355,566.82000000000005002,-55.24000000000000199,0.0,35.68999999999999773,8.19999999999999929,0.65100000000000002,G3V,1,1,NaN


In [328]:
montu_catalogue[montu_catalogue.HD == 138917]

,MN,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,...,pmRA,pmDec,RadVel,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,UpdateName
413,414,138917,5788,0,HD 138917,13Del Ser/HD 138917/HR 5788/MN 414,NaN,NaN,NaN,NaN,...,-74.00000000000000000,-5.00000000000000000,-38.0,21.00000000000000000,3.79999999999999982,0.26000000000000001,F0IV,1,0,NaN
414,415,138917,5789,76276,δ Ser,13 Ser/13Del Ser/BD B+11 2821 /HD 138917/HD 13...,NaN,δ Ser,13 Ser,Ser,...,-72.62999999999999545,3.16999999999999993,-42.0,15.53999999999999915,3.79999999999999982,0.26800000000000002,F0IV,1,1,NaN


In [335]:
id = 'HR'
cond = (montu_catalogue[id] > 0)&(montu_catalogue[id].duplicated())
repeated = montu_catalogue[cond]
cond.sum()
Montu.print_df(repeated)

,MN,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,RAJ2000,DecJ2000,pmRA,pmDec,RadVel,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,UpdateName
115,116,129989,5506,0,HD 129989,36Eps Boo/HD 129989/HR 5506/MN 116,NaN,NaN,NaN,NaN,221.24666666666664128,27.07416666666666671,-49.00000000000000000,21.00000000000000000,-17.0,16.00000000000000000,2.70000000000000018,0.96999999999999997,K0-II-III,1,0,NaN
497,498,17878,854,0,HD 17878,18Tau Per/HD 17878/HR 854/MN 498,NaN,NaN,NaN,NaN,43.56458333333333144,52.76250000000000284,0.00000000000000000,-5.00000000000000000,2.0,19.00000000000000000,3.95000000000000018,0.73999999999999999,G4III+A4V,1,1,NaN
518,519,192909,7751,0,HD 192909,32 Cyg/HD 192909/HR 7751/MN 519,NaN,NaN,NaN,NaN,303.86791666666670153,47.71444444444444599,1.00000000000000000,8.00000000000000000,-14.0,14.00000000000000000,3.97999999999999998,1.52000000000000002,K3Ib+B3V,1,1,NaN
754,755,12447,596,0,HD 12447,113Alp Psc/HD 12447/HR 596/MN 755,NaN,NaN,NaN,NaN,30.51166666666666671,2.76361111111111102,33.00000000000000000,-5.00000000000000000,9.0,5.00000000000000000,4.33000000000000007,0.03000000000000000,A0pSiSr,1,1,NaN
801,802,213310,8572,0,HD 213310,5 Lac/HD 213310/HR 8572/MN 802,NaN,NaN,NaN,NaN,337.38249999999999318,47.70694444444444571,2.00000000000000000,0.00000000000000000,-4.0,6.00000000000000000,4.36000000000000032,1.67999999999999994,M0II+B8V,0,1,NaN
1168,1169,218640,8817,114375,c3 Aqr,89 Aqr/HD 218640/HIP 114375/MN 1169/RST 3320/c3 Aqr,NaN,NaN,89 Aqr,Aqr,347.47866666666664059,-22.45758333333332857,35.92000000000000171,-7.79000000000000004,0.0,6.25999999999999979,4.70999999999999996,0.67400000000000004,A3IV:,1,1,NaN
1835,1836,25007,1230,0,HD 25007,HD 25007/HR 1230/MN 1836,NaN,NaN,NaN,NaN,62.51166666666667027,80.69861111111111995,-17.00000000000000000,-2.00000000000000000,4.0,5.00000000000000000,5.09999999999999964,0.56000000000000005,G8III+A6V,1,0,NaN
2087,2088,219834,8866,115126,94 Aqr A,94 Aqr/94 Aqr A/BD B-14 6448 /HD 219834/HIP 115126/HR 8866/MCA 74/MN 2088/Σ 2998,NaN,NaN,NaN,NaN,349.77712500000001228,-13.45844444444444399,265.63999999999998636,-44.86999999999999744,10.0,48.21999999999999886,5.20000000000000018,0.78700000000000003,G5IV,1,1,NaN
2336,2337,206487,8289,0,HD 206487,7 Peg/HD 206487/HR 8289/MN 2337,NaN,NaN,NaN,NaN,325.56458333333330302,5.68000000000000060,14.00000000000000000,-6.00000000000000000,-4.0,0.00000000000000000,5.29999999999999982,1.63999999999999990,M2IIIab,0,1,NaN
2776,2777,25555,1252,0,HD 25555,36 Tau/HD 25555/HR 1252/MN 2777,NaN,NaN,NaN,NaN,61.09041666666666259,24.10583333333333300,-4.00000000000000000,-14.00000000000000000,18.0,14.00000000000000000,5.46999999999999975,0.85999999999999999,G0III+A4V,1,0,NaN


In [337]:
Montu.print_df(montu_catalogue[montu_catalogue.HR == 8526])

,MN,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,RAJ2000,DecJ2000,pmRA,pmDec,RadVel,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,UpdateName
5813,5814,212168,8526,110712,Δ 238,HD 212168/HIP 110712/HR 8526/KO 5/MN 5814/Δ 238,NaN,NaN,NaN,NaN,336.46262500000005957,-75.01572222222222308,58.54999999999999716,12.48000000000000043,14.0,43.39000000000000057,6.12000000000000011,0.59899999999999998,G3V,1,1,NaN
24895,24896,212168,8526,110719,TOK 434,HD 212168/HIP 110719/HR 8526/MN 24896/TOK 434,NaN,NaN,NaN,NaN,336.48437500000000000,-75.01469444444444434,15.17999999999999972,61.32000000000000028,14.0,60.24000000000000199,9.34999999999999964,1.12000000000000011,G3V,1,1,NaN


### HYG Catalogue

This will the the official source of `MontuPython`: 

- WebPage: http://www.astronexus.com/hyg, 
- GitHub: https://github.com/astronexus/HYG-Database/tree/master/hyg 

In [350]:
Montu._wget('https://raw.githubusercontent.com/astronexus/HYG-Database/master/hyg/v3/hyg_v37.csv',filename='catalogues/hyg_v37.csv')

32.4MB [00:00, 60.1MB/s]                            


In [354]:
hyg_cat = pd.read_csv('catalogues/hyg_v37.csv')
hyg_cat.sort_values(by='mag',inplace=True)

In [356]:
Montu.print_df(hyg_cat.head(100))

,id,hip,hd,hr,gl,bf,proper,ra,dec,dist,pmra,pmdec,rv,mag,absmag,spect,ci,x,y,z,vx,vy,vz,rarad,decrad,pmrarad,pmdecrad,bayer,flam,con,comp,comp_primary,base,lum,var,var_min,var_max
0,0,NaN,NaN,NaN,NaN,NaN,Sol,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,-26.69999999999999929,4.84999999999999964,G2V,0.65600000000000003,0.00000500000000000,0.00000000000000000,0.00000000000000000,0.00000000,0.00000000,0.00000000,0.00000000000000000,0.00000000000000000,0.0000000000000000,0.000000000000000,NaN,NaN,NaN,1,0,NaN,1.00000000000000000,NaN,NaN,NaN
32263,32263,32349.0,48915.0,2491.0,Gl 244A,9Alp CMa,Sirius,6.75248100000000040,-16.71611599999999953,2.63710000000000022,-546.00999999999999091,-1223.07999999999992724,-9.40000000000000036,-1.43999999999999995,1.45399999999999996,A0m...,0.00900000000000000,-0.49432300000000001,2.47673100000000002,-0.75848499999999996,0.00000953,-0.00001207,-0.00001221,1.76779536960219952,-0.29175125851768502,-0.0000026471311772,-0.000005929659164,Alp,9.0,CMa,1,32263,Gl 244,22.82443312173503713,NaN,-1.33299999999999996,-1.52299999999999991
30365,30365,30438.0,45348.0,2326.0,NaN,Alp Car,Canopus,6.39919499999999974,-52.69565999999999661,94.78669999999999618,19.98999999999999844,23.67000000000000171,21.00000000000000000,-0.62000000000000000,-5.50399999999999956,F0Ib,0.16400000000000001,-5.99267899999999987,57.13203399999999732,-75.39610500000000570,-0.00001140,0.00002059,-0.00001049,1.67530535199972919,-0.91971277489026759,0.0000000969142547,0.000000114755398,Alp,NaN,Car,1,30365,NaN,13854.79167566721662297,NaN,-0.54700000000000004,-0.66700000000000004
69450,69451,69673.0,124897.0,5340.0,Gl 541,16Alp Boo,Arcturus,14.26102999999999987,19.18241000000000085,11.25750000000000028,-1093.45000000000004547,-1999.40000000000009095,-5.00000000000000000,-0.05000000000000000,-0.30700000000000000,K2IIIp,1.23900000000000010,-8.82344800000000085,-5.93253100000000000,3.69893800000000006,-0.00005905,0.00003220,-0.00010473,3.73352902854208724,0.33479621805862120,-0.0000053011951900,-0.000009693364729,Alp,16.0,Boo,1,69451,NaN,115.55799554021291442,NaN,NaN,NaN
71455,71456,71683.0,128620.0,5459.0,Gl 559A,Alp1Cen,Rigil Kentaurus,14.66076499999999960,-60.83397599999999983,1.32479999999999998,-3678.19000000000005457,481.83999999999997499,-26.19999999999999929,-0.01000000000000000,4.37899999999999956,G2V,0.70999999999999996,-0.49520300000000000,-0.41408400000000001,-1.15662500000000001,-0.00000722,0.00002476,0.00002491,3.83817921177722798,-1.06175317577041750,-0.0000178323683168,0.000002336026238,Alp-1,NaN,Cen,1,71456,Gl 559,1.54312106565779206,NaN,0.11300000000000000,-0.08699999999999999
90978,90979,91262.0,172167.0,7001.0,Gl 721,3Alp Lyr,Vega,18.61563999999999908,38.78369200000000205,7.67870000000000008,201.02000000000001023,287.45999999999997954,-12.09999999999999964,0.03000000000000000,0.60399999999999998,A0Vvar,-0.00100000000000000,0.96056500000000000,-5.90800899999999984,4.80973100000000020,0.00000476,0.00001734,0.00000059,4.87356309550903077,0.67690311639730250,0.0000009745724607,0.000001393645406,Alp,3.0,Lyr,1,90979,NaN,49.93441887213497665,NaN,NaN,NaN
24549,24549,24608.0,34029.0,1708.0,Gl 194A,13Alp Aur,Capella,5.27815000000000012,45.99799099999999896,13.12340000000000018,75.51999999999999602,-427.12999999999999545,22.19999999999999929,0.08000000000000000,-0.51000000000000001,M1: comp,0.79500000000000004,1.71263300000000007,8.95443900000000070,9.44000699999999959,0.00000192,0.00003559,-0.00000254,1.38181641264880106,0.80281639263411120,0.0000003661312916,-0.000002070784673,Alp,13.0,Aur,1,24549,Gl 194,139.31568029453021040,NaN,NaN,NaN
24378,24378,24436.0,34085.0,1713.0,NaN,19Bet Ori,Rigel,5.24229799999999990,-8.20163999999999938,264.55029999999999291,1.87000000000000011,-0.56000000000000005,21.00000000000000000,0.17999999999999999,-6.93299999999999983,B8Ia,-0.03000000000000000,51.60110600000000147,256.70990499999999201,-37.74005100000000112,0.00000182,0.0000212

In [359]:
print("Catalogue completeness:")
nstars = len(hyg_cat)
columns = hyg_cat.columns
for col in columns:
    ncomplete = (~(hyg_cat[col].isnull())).sum()
    print(f"\t{col}: {ncomplete}/{nstars} ({ncomplete/nstars*100:.1f}%)")

Catalogue completeness:
	id: 119626/119626 (100.0%)
	hip: 117951/119626 (98.6%)
	hd: 98885/119626 (82.7%)
	hr: 9041/119626 (7.6%)
	gl: 3801/119626 (3.2%)
	bf: 3099/119626 (2.6%)
	proper: 367/119626 (0.3%)
	ra: 119626/119626 (100.0%)
	dec: 119626/119626 (100.0%)
	dist: 119626/119626 (100.0%)
	pmra: 119626/119626 (100.0%)
	pmdec: 119626/119626 (100.0%)
	rv: 119626/119626 (100.0%)
	mag: 119626/119626 (100.0%)
	absmag: 119626/119626 (100.0%)
	spect: 116578/119626 (97.5%)
	ci: 117735/119626 (98.4%)
	x: 119626/119626 (100.0%)
	y: 119626/119626 (100.0%)
	z: 119626/119626 (100.0%)
	vx: 119626/119626 (100.0%)
	vy: 119626/119626 (100.0%)
	vz: 119626/119626 (100.0%)
	rarad: 119626/119626 (100.0%)
	decrad: 119626/119626 (100.0%)
	pmrarad: 119626/119626 (100.0%)
	pmdecrad: 119626/119626 (100.0%)
	bayer: 1537/119626 (1.3%)
	flam: 2737/119626 (2.3%)
	con: 119625/119626 (100.0%)
	comp: 119626/119626 (100.0%)
	comp_primary: 119626/119626 (100.0%)
	base: 1086/119626 (0.9%)
	lum: 119626/119626 (100.0%)
	

In [357]:
Montu.print_df(montu_catalogue.head(100))

,MN,HD,HR,HIP,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,RAJ2000,DecJ2000,pmRA,pmDec,RadVel,Parallax,Vmag,B-V,SpType,IsMultiple,IsVariable,UpdateName
0,1,48915,2491,32349,Sirius,9 CMa/9Alp CMa/AGC 1/BD B-16 1591 /HD 48915/HIP 32349/HL 3/HR 2491/LEC 1/MN 1/α CMa/β 1411,Sirius,α CMa,9 CMa,CMa,101.28854166666666003,-16.71313888888888854,-546.00999999999999091,-1223.07999999999992724,-8.0,379.20999999999997954,-1.43999999999999995,0.00900000000000000,A1Vm,1,1,2016-06-30
1,2,45348,2326,30438,Canopus,Alp Car/CRU 9006/HD 45348/HIP 30438/HR 2326/MAM 2/MN 2/α Car,Canopus,α Car,NaN,Car,95.98787500000000250,-52.69572222222221569,19.98999999999999844,23.67000000000000171,21.0,10.42999999999999972,-0.62000000000000000,0.16400000000000001,F0II,0,1,2016-06-30
2,3,124897,5340,69673,Arcturus,16 Boo/16Alp Boo/BD B+19 2777 /HD 124897/HIP 69673/HR 5340/MN 3/α Boo,Arcturus,α Boo,16 Boo,Boo,213.91812500000000341,19.18727777777777632,-1093.45000000000004547,-1999.40000000000009095,-5.0,88.84999999999999432,-0.05000000000000000,1.23900000000000010,K1.5IIIFe-0.5,1,1,2016-06-30
3,4,128620,5459,71683,Rigil Kentaurus,Alp1Cen/HD 128620/HIP 71683/HR 5459/LDS 494/MN 4/RHD 1/α Cen/α1 Cen,Rigil Kentaurus,α1 Cen,NaN,Cen,219.92041666666665378,-60.83513888888889198,-3678.19000000000005457,481.83999999999997499,-22.0,742.12000000000000455,-0.01000000000000000,0.70999999999999996,G2V,1,1,2016-11-06
4,5,172167,7001,91262,Vega,3 Lyr/3Alp Lyr/BD B+38 3238 /H 5 39/HD 172167/HIP 91262/HR 7001/MBA 2/MET 11/MN 5/SMR 33/Σ II 9/α Lyr,Vega,α Lyr,3 Lyr,Lyr,279.23412500000000591,38.78300000000000125,201.02000000000001023,287.45999999999997954,-14.0,128.93000000000000682,0.03000000000000000,-0.00100000000000000,A0Va,0,1,2016-06-30
5,6,34029,1708,24608,Capella,13 Aur/13Alp Aur/ANJ 1/BAR 25/BD B+45 1077 /FRH 1/HD 34029/HIP 24608/HJ 2256/HR 1708/MN 6/SHJ 51/SHY 23/SMR 26/α Aur/β 1392,Capella,α Aur,13 Aur,Aur,79.17208333333333314,45.99902777777777629,75.51999999999999602,-427.12999999999999545,30.0,77.29000000000000625,0.08000000000000000,0.79500000000000004,G5IIIe+G0III,1,1,2016-06-30
6,7,34085,1713,24436,Rigel,19 Ori/19Bet Ori/BD B-08 1063 /HD 34085/HIP 24436/HR 1713/MN 7/Σ 668/β 555/β Ori,Rigel,β Ori,19 Ori,Ori,78.63445833333334178,-8.20163888888888870,1.87000000000000011,-0.56000000000000005,21.0,4.21999999999999975,0.17999999999999999,-0.03000000000000000,B8Ia:,1,1,2016-06-30
7,8,61421,2943,37279,Procyon,10 CMi/10Alp CMi/BD B+05 1739 /D 29/DIC 1/HD 61421/HIP 37279/HR 2943/LAM 6/MN 8/SHB 1/SLE 439/SMR 11/α CMi,Procyon,α CMi,10 CMi,CMi,114.82725000000000648,5.22750000000000004,-716.57000000000005002,-1034.57999999999992724,-3.0,285.93000000000000682,0.40000000000000002,0.43200000000000000,F5IV-V,1,0,2016-06-30
8,9,10144,472,7588,Achernar,Alp Eri/HD 10144/HIP 7588/HR 472/MN 9/α Eri,Achernar,α Eri,NaN,Eri,24.42812500000000142,-57.23666666666666458,88.01999999999999602,-40.07999999999999829,16.0,22.67999999999999972,0.45000000000000001,-0.15800000000000000,B3Vpe,1,1,2016-06-30
9,10,39801,2061,27989,Betelgeuse,58 Ori/58Alp Ori/BD B+07 1055 /H 6 39/HD 39801/HIP 27989/HR 2061/KAR 1/MN 10/SLE 831/SMR 29/α Ori,Betelgeuse,α Ori,58 Ori,Ori,88.79287500000000932,7.40702777777777843,27.32999999999999829,10.85999999999999943,21.0,7.62999999999999989,0.45000000000000001,1.50000000000000000,M1-2Ia-Iab,1,1,2016-06-30
